## Caricamento dei file contenenti il dataset

In [1]:
import pandas as pd

# Load File and create a new dataframe called 'df'
df = pd.read_csv('full_tagged_dataset_2%.csv')

# Fare merge dei 5 file che contengono l'intero dataset

df

,TITLE,NER,difficulty,time,cost,method,INGREDIENTS,DIRECTIONS,total_time,PREPARATION_TIME,NER_clean,VEGAN,NON_VEGAN_INGREDIENTS,VEGETARIAN,NON_VEGETARIAN_INGREDIENTS,total_cost,price_tag,categories,vegan,vegetarian
0,Western Sizzlin Bread Pudding,"['eggs', 'milk', 'sugar', 'vanilla', 'gallon w...",Hard,Slow,Cheap,Bake,"[""16 eggs, beaten"", ""6 -8 evaporated milk, 360...","[""Distribute rolls and cinnamon roll in (4) 2\...",150.0,Very slow (90+ mins),"['eggs', 'milk', 'sugar', 'vanilla', 'gallon w...",False,"['eggs', 'milk']",True,[],49.2,medium,"['seafood', 'dairy', 'sweetener', 'spice/herb'...",False,False
1,Creamy Tuna And Bacon Salad (Reduced Fat),"['tuna', 'mayonnaise', 'low-fat sour cream', '...",Hard,Average,Cheap,Other,"[""1 (6 ounce) can tuna, drained and flaked"", ""...","[""Put everything in a bowl and mix together un...",15.0,Fast (10-20 mins),"['tuna', 'mayonnaise', 'low-fat sour cream', '...",False,"['tuna', 'low-fat sour cream', 'bacon bits']",False,"['tuna', 'bacon bits']",52.5,expensive,"['seafood', 'condiment', None, 'meat', 'spice/...",False,False
2,Spinach And Mushroom Lasagna,"['olive oil', 'onion', 'garlic', 'salt', 'grou...",Hard,Quick,Cheap,Bake,"[""1 tablespoon olive oil"", ""1 medium onion , c...","[""Heat oven to 375 degrees."", """", ""In medium s...",70.0,Slow (40-90 mins),"['olive oil', 'onion', 'garlic', 'salt', 'grou...",False,"['ricotta cheese', 'Parmesan cheese', 'egg']",True,[],78.0,rich,"['condiment', 'vegetable', 'vegetable', 'condi...",False,False
3,Three-Bean Tacos,"['olive oil', 'onion', 'red bell pepper', 'gre...",Hard,Quick,Cheap,Boil,"[""1 teaspoon olive oil"", ""1 cup diced onion"", ...","[""Heat oil in a large skillet over medium-high...",22.0,Medium (20-40 mins),"['olive oil', 'onion', 'red bell pepper', 'gre...",False,['cheddar cheese'],True,[],87.5,rich,"['condiment', 'vegetable', 'vegetable', 'veget...",False,False
4,Hearty Hamburger Soup,"['lean ground beef', 'white onion', 'ground bl...",Hard,Quick,Expensive,Boil,"[""900.0 gs lean ground beef"", ""1 white onion, ...","[""Brown ground beef and onion in a large pot. ...",45.0,Slow (40-90 mins),"['lean ground beef', 'white onion', 'ground bl...",False,"['lean ground beef', 'parmesan cheese', 'sour ...",False,['lean ground beef'],96.3,rich,"['meat', 'vegetable', 'condiment', 'spice/herb...",False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44618,California Chili Powder Recipe,"['Mexico chilies', 'grnd cumin', 'cayenne', 'o...",Easy,Average,Cheap,Other,"[""0.25 c. grnd Calif or possibly New Mexico ch...","[""In a 0.3333333333- to 0.5-c. tall, narrow cl...",30.0,Medium (20-40 mins),"['Mexico chilies', 'grnd cumin', 'cayenne', 'o...",True,[],True,[],78.8,rich,"[None, 'spice/herb', 'spice/herb', 'spice/herb...",True,True
44619,Pizza Margherita,"['batch pizza', 'marinara sauce', 'mozzarella'...",Easy,Quick,Cheap,Bake,"[""1 batch pizza dough (see preceding recipe)"",...","[""Preheat oven to 450 degrees F. Place a pizza...",10.0,Fast (10-20 mins),"['batch pizza', 'marinara sauce', 'mozzarella'...",True,[],True,[],18.0,very cheap,"['processed', 'condiment', 'dairy', None]",False,True
44620,Frenchish Chicken And Red Wine Casserole,"['tomatoes', 'red wine', 'chicken stock', 'gar...",Hard,Slow,Cheap,Boil,"[""1 (14 ounce) can chopped tomatoes"", ""1.25 cu...","[""Place the tomatoes, wine, stock, garlic, ros...",100.0,Very slow (90+ mins),"['tomatoes', 'red wine', 'chicken stock', 'gar...",False,"['chicken stock', 'chicken thighs', 'chicken']",False,"['chicken stock', 'chicken thighs', 'chicken']",60.3,expensive,"['vegetable', 'processed', 'processed', 'veget...",False,False
44621,Blueberry Almond Streusel Muffins Recipe,"['flour', 'sugar', 'baking pwdr', 'baking soda...",Hard,Average,Cheap,Bake,"[""2.5 c. all-purpose flour"", ""1 c. granulated ...","[""Preheat oven to 400 ."", ""Combine first 5 ing...",18.0,Fast (10-20 mins),"['flour', 'sugar', 'baking pwdr', 'baking soda...",False,"['milk', 'low-

## Definizione dei metodi per le conversioni di frazioni e unità di misura e applicazione di essi sul dataframe

In [4]:
from fractions import Fraction


# === UTILITY FUNCTIONS ===

def convert_fractions(text):
    """Convert fractions in the format '1 1/2' or '3/4' to decimals (e.g., '1.5', '0.75')."""
    # Match optional whole number + fraction, or just fraction
    pattern = r'(?:(\d+)\s+)?(\d+)/(\d+)'

    def replacer(match):
        whole = int(match.group(1)) if match.group(1) else 0
        numerator = int(match.group(2))
        denominator = int(match.group(3))
        decimal = whole + Fraction(numerator, denominator)
        return "{:.10g}".format(float(decimal))  # Clean, no trailing zeros

    return re.sub(pattern, replacer, text)


# Fixed conversions
CONVERSIONS = {
    "oz": 30,      # ounce → gram
    "lb": 450,      # pounds → gram
    "pt": 475,      # pint → milliliter
    "qt": 950,      # quart → milliliter
    "inch": 2.5        # inches → centimeter
}

def unit_to_metric(unit):
    """Return the metric equivalent of the given imperial unit."""
    return {
        "oz": "g",
        "lb": "g",
        "pt": "ml",
        "qt": "ml",
        "inch": "cm"
    }[unit]

def convert_units(text):
    """Convert imperial units (oz, lb, pt, qt, inch) to metric units in the text."""
    if pd.isna(text):
        return text

    # Convert oz, lb, pt, qt (form: "4 oz", "2.5 lb", etc.)
    for unit, factor in CONVERSIONS.items():
        pattern = r'(\d+(\.\d+)?)\s*' + unit
        text = re.sub(pattern, lambda m: f"{round(float(m.group(1)) * factor, 1)} {unit_to_metric(unit)}", text)

    # Convert measure in inches in pans
    text = re.sub(r'(\d+)\s*x\s*(\d+)[-\s]*inch', lambda
        m: f"{int(m.group(1)) * CONVERSIONS['inch']:.0f} x {int(m.group(2)) * CONVERSIONS['inch']:.0f} cm", text)

    return text

def convert_fahrenheit_to_celsius(text):
    """Convert Fahrenheit temperatures to Celsius (e.g., 350°F → 177°C)."""
    # Match degrees like 275°, 275 °F, 275\u00b0F (escaped), etc.
    pattern = r'(\d+)\s*(?:°|\\u00b0|degrees)'

    def replacer(match):
        fahrenheit = int(match.group(1))
        celsius = round((fahrenheit - 32) * 5 / 9)
        return f"{celsius}°C"

    return re.sub(pattern, replacer, text)

# righe non utilizzabili perche le colonne non sono presenti nel dataset gia taggato, risolvere come ti avevo gia detto
'''
# === APPLY TRANSFORMATIONS ===

# Convert fractions and units in ingredients and directions
df['INGREDIENTS'] = df['ingredients'].apply(convert_fractions).apply(convert_units)
df['DIRECTIONS'] = df['directions'].apply(convert_fractions).apply(convert_units).apply(convert_fahrenheit_to_celsius)

# Rename title column and drop raw ingredients/directions columns
df = df.rename(columns={"title": "TITLE"}).drop(columns=["ingredients", "directions"])
'''

'\n# === APPLY TRANSFORMATIONS ===\n\n# Convert fractions and units in ingredients and directions\ndf[\'INGREDIENTS\'] = df[\'ingredients\'].apply(convert_fractions).apply(convert_units)\ndf[\'DIRECTIONS\'] = df[\'directions\'].apply(convert_fractions).apply(convert_units).apply(convert_fahrenheit_to_celsius)\n\n# Rename title column and drop raw ingredients/directions columns\ndf = df.rename(columns={"title": "TITLE"}).drop(columns=["ingredients", "directions"])\n'

## Definizione di una euristica in grado di categorizzare il tempo di preparazione di una ricetta

In [5]:
import re

# Patterns to detect time expressions
TIME_PATTERN = re.compile(
    r'(\d+(?:\.\d+)?)\s*(minute|minutes|min|mins|m|hour|hours|hr|hrs|h|day|days|d)\b',
    re.IGNORECASE
)

RANGE_PATTERN = re.compile(
    r'(\d+)\s*-\s*(\d+)\s*(minute|minutes|min|mins|m|hour|hours|hr|hrs|h)\b',
    re.IGNORECASE
)

# Time estimates for common preparation methods
PREP_ESTIMATES = {
    'bake': 45, 'boil': 20, 'fry': 15, 'grill': 25, 'chill': 120,
    'simmer': 30, 'marinate': 60, 'microwave': 10, 'no-bake': 20,
    'refrigerate': 180, 'freeze': 240
}

# Special cases for recipe types
RECIPE_TYPE_ESTIMATES = {
    'pasta': 10, 'salad': 15, 'cake': 45, 'pie': 60, 'stew': 120,
    'casserole': 60, 'soup': 30, 'cookies': 30, 'bread': 90,
    'fudge': 20, 'candy': 30
}

def convert_to_minutes(qty, unit):
    """Converts time quantity to minutes."""
    qty = float(qty)
    unit = unit.lower()
    if unit.startswith('m'): return qty
    if unit.startswith('h'): return qty * 60
    if unit.startswith('d'): return qty * 1440
    return 0

def estimate_by_recipe_type(title, ingredients):
    """Fallback estimation based on recipe title or ingredients."""
    title = str(title).lower()
    ingredients = str(ingredients).lower()

    # Check for specific recipe types
    for keyword, est in RECIPE_TYPE_ESTIMATES.items():
        if keyword in title:
            return est

    if any(word in ingredients for word in ['raw', 'fresh']):
        return 15
    if 'frozen' in ingredients:
        return 20
    if 'canned' in ingredients:
        return 10

    # Default fallback
    return 30

def clean_instruction_step(step):
    """Normalize range formats and identify special cases."""
    # Replace ranges (e.g., "10-15 minutes" → "15 minutes")
    step = RANGE_PATTERN.sub(lambda m: f"{m.group(2)} {m.group(3)}", step)

    # Handle special keywords
    lowered = step.lower()
    if 'overnight' in lowered:
        return 480
    if 'until set' in lowered or 'until firm' in lowered:
        return 60

    # Sum all time expressions
    return sum(
        convert_to_minutes(qty, unit)
        for qty, unit in TIME_PATTERN.findall(step)
    )

def parse_instructions(instructions):
    """Parses instructions and extracts total estimated time in minutes."""
    total_time = 0

    if not isinstance(instructions, list):
        try:
            instructions = ast.literal_eval(str(instructions))
        except:
            instructions = [instructions]

    for step in instructions:
        if isinstance(step, str):
            total_time += clean_instruction_step(step)

    return total_time

def categorize_time(total_time):
    """Classify total time into labeled categories."""
    if total_time == 0:
        return 'Not specified'
    if total_time < 10:
        return 'Very fast (0-10 mins)'
    if total_time < 20:
        return 'Fast (10-20 mins)'
    if total_time < 40:
        return 'Medium (20-40 mins)'
    if total_time < 90:
        return 'Slow (40-90 mins)'
    return 'Very slow (90+ mins)'

# Calculate total preparation time
df['total_time'] = df['DIRECTIONS'].apply(parse_instructions)

# Estimate time for unspecified recipes
missing = df['total_time'] == 0
df.loc[missing, 'total_time'] = df[missing].apply(
    lambda row: estimate_by_recipe_type(row['TITLE'], row['INGREDIENTS']),
    axis=1
)

# Categorize preparation times
df['PREPARATION_TIME'] = df['total_time'].apply(categorize_time)

df[['TITLE', 'INGREDIENTS', 'DIRECTIONS', 'PREPARATION_TIME']].head(200)

,TITLE,INGREDIENTS,DIRECTIONS,PREPARATION_TIME
0,Western Sizzlin Bread Pudding,"[""16 eggs, beaten"", ""6 -8 evaporated milk, 360...","[""Distribute rolls and cinnamon roll in (4) 2\...",Very slow (90+ mins)
1,Creamy Tuna And Bacon Salad (Reduced Fat),"[""1 (6 ounce) can tuna, drained and flaked"", ""...","[""Put everything in a bowl and mix together un...",Fast (10-20 mins)
2,Spinach And Mushroom Lasagna,"[""1 tablespoon olive oil"", ""1 medium onion , c...","[""Heat oven to 375 degrees."", """", ""In medium s...",Slow (40-90 mins)
3,Three-Bean Tacos,"[""1 teaspoon olive oil"", ""1 cup diced onion"", ...","[""Heat oil in a large skillet over medium-high...",Medium (20-40 mins)
4,Hearty Hamburger Soup,"[""900.0 gs lean ground beef"", ""1 white onion, ...","[""Brown ground beef and onion in a large pot. ...",Slow (40-90 mins)
...,...,...,...,...
195,Sweet Potato Brotchen (Bread Rolls),"[""450 g sweet potatoes"", ""15 g butter or olive...","[""Peel and cut sweet potatoes into chunks."", ""...",Very slow (90+ mins)
196,Green Cleaner and Bug Repellant in One (Concen...,"[""1 tablespoon essential oil (part)"", ""3 table...","[""Mix all ingredients."", ""Store in a glass jar...",Medium (20-40 mins)
197,Chocolate Chip Pie,"[""1 graham cracker crust"", ""20 large marshmall...","[""Make crust (I buy the prepared ones from the...",Slow (40-90 mins)
198,Poppy Seed Dressing,"[""0.75 c. sugar"", ""2 tsp. dry mustard"", ""2 tsp...","[""Combine first 6 ingredients in a blender con...",Very fast (0-10 mins)


## La cella sottostante contiene una logica per calcolare la categoria di costo di una ricetta, che è stata poi sostituita da un metodo più efficace

In [ ]:
'''
# Conversione unità → kg
UNIT_CONVERSION = {
    'cup': 0.24, 'c.': 0.24, 'c': 0.24,
    'teaspoon': 0.005, 'tsp': 0.005, 'tsp.': 0.005,
    'tablespoon': 0.015, 'tbsp': 0.015, 'tbsp.': 0.015,
    'package': 0.2, 'pkg': 0.2, 'pkg.': 0.2,
    'can': 0.4, 'carton': 1.0,
    'g': 0.001, 'g.': 0.001, 'gram': 0.001,
    'kg': 1.0, 'kilogram': 1.0,
    'lb': 0.4536, 'pound': 0.4536,
    'oz': 0.02835, 'ounce': 0.02835,
    'large package': 0.5, 'large pkg.': 0.5, 'large pkg': 0.5
}

# 3. Parsing function (qty, unit, name, grams_in_paren)
def parse_ingredient(ing_str):
    # peso fra parentesi
    m = re.search(r'\(\s*([0-9]+(?:\.[0-9]+)?)\s*g\.?\s*\)', ing_str, re.IGNORECASE)
    grams = float(m.group(1)) if m else None

    # quantity e unit
    patt = r'^\s*([\d\/.\s]+)?\s*([a-zA-Z\.]+)?'
    m2 = re.match(patt, ing_str)
    qty_raw = m2.group(1) if m2 else None
    try:
        qty = eval(qty_raw.replace(' ', '+')) if qty_raw else 1.0
    except:
        qty = 1.0
    unit = (m2.group(2) or '').strip('.').lower() if m2 else ''

    # name pulito
    name = re.sub(r'\(.*?\)|optional', '', ing_str, flags=re.IGNORECASE)
    # rimuovo quantità/unità
    name = re.sub(r'^[\d\/.\s]+\s*[a-zA-Z\.]*', '', name).strip().lower()
    return qty, unit, name, grams

# 4. Calcolo posizionale con fallback substring e token
def calculate_recipe_cost_positional(ingredients_list, ner_list):
    total = 0.0
    missing = []

    for ing_str, ner_item in zip(ingredients_list, ner_list):
        key = ner_item.lower()
        qty, unit, name, grams = parse_ingredient(ing_str)

        # scorporo se passo i grams
        if grams is not None:
            kg = (grams / 1000) * qty
        elif unit == '':  # a pezzo
            # prendo prezzo direct key
            price = price_dict.get(key)
            # fallback substring/token
            if price is None:
                # substring match
                for k, v in price_dict.items():
                    if k in key or key in k:
                        price = v
                        break
                # token match
                if price is None:
                    for token in key.split():
                        if token in price_dict:
                            price = price_dict[token]
                            break
            if price:
                total += qty * price
            else:
                missing.append(key)
            continue
        else:
            conv = UNIT_CONVERSION.get(unit, 0.0)
            if conv == 0:
                missing.append(key)
                continue
            kg = qty * conv

        # prezzo per kg
        price = price_dict.get(key)
        if price is None:
            for k, v in price_dict.items():
                if k in key or key in k:
                    price = v
                    break
        if price is None:
            missing.append(key)
            continue

        total += kg * price

    return round(total, 2), missing

# 5. Applica al DataFrame
    df['INGREDIENTS'] = df['INGREDIENTS'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    df['NER_clean']  = df['NER_clean'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

def compute_cost_row(row):
     cost, _ = calculate_recipe_cost_positional(row['INGREDIENTS'], row['NER_clean'])
     return cost

df['total_cost'] = df.apply(compute_cost_row, axis=1)

df['CATEGORY_COST'] = df['total_cost'].apply(categorize_cost)

# Mostra risultati
df[['INGREDIENTS', 'NER', 'CATEGORY_COST']].head(200)'''

## Le celle successive sono in grado di determinare se una ricetta appartiene alle categorie 'vegan' o 'vegetarian'

In [10]:
import ast
import re

# Cleaning the NER column from special symbols
df['NER_clean'] = df['NER'].apply(ast.literal_eval)

# List containing keywords
NON_VEGAN_KEYWORDS = {
    # meat
    'meat', 'red meat', 'white meat', 'poultry', 'veal', 'lamb', 'duck',
    'goose', 'chicken', 'beef', 'pork', 'bacon', 'ham', 'prosciutto',
    'sausage', 'salami', 'hot dog', 'pepperoni', 'mortadella', 'chorizo',
    'ground beef', 'minced meat', 'steak', 'ribs', 'brisket', 'lardons',

    # fish or seafood
    'fish', 'anchovy', 'anchovies', 'tuna', 'salmon', 'sardines', 'mackerel',
    'cod', 'haddock', 'tilapia', 'trout', 'snapper', 'catfish', 'bass',
    'shellfish', 'shrimp', 'prawns', 'crab', 'lobster', 'scampi', 'clam',
    'mussels', 'oysters', 'scallops', 'caviar', 'roe',

    # dairy
    'milk', 'whole milk', 'skim milk', 'cheese', 'butter', 'cream',
    'heavy cream', 'sour cream', 'whipped cream', 'yogurt', 'ghee',
    'lactose', 'casein', 'whey', 'buttermilk', 'condensed milk',
    'evaporated milk', 'parmesan', 'mozzarella', 'cheddar',
    'shredded cheese', 'blue cheese', 'goat cheese', 'cream cheese',
    'cottage cheese', 'ricotta', 'cocoa butter',

    # eggs
    'egg', 'eggs', 'egg yolk', 'egg white', 'albumen', 'mayonnaise',

    # bees
    'honey', 'royal jelly', 'propolis', 'bee pollen',

    # animal derivatives
    'gelatin', 'gelatina', 'collagen', 'isinglass', 'lard', 'suet',
    'carmine', 'shellac', 'pepsin', 'bone broth', 'bone marrow',
    'broth (meat)', 'stock (meat)', 'chicken broth', 'beef broth',
    'animal fat'
}

VEGAN_EXCEPTIONS = {
    'milk': {'soy', 'almond', 'oat', 'rice', 'coconut', 'cashew', 'hazelnut',
             'hemp', 'pea', 'flax', 'macadamia', 'barley', 'plant-based milk'},
    'cheese': {'vegan cheese', 'nutritional yeast', 'cashew cheese', 'tofu cheese',
               'almond cheese', 'plant-based cheese'},
    'meat': {'soy', 'seitan', 'tofu', 'tempeh', 'jackfruit', 'plant-based', 'textured soy protein', 'mushroom-based'},
    'butter': {'vegan butter', 'plant butter', 'peanut butter', 'almond butter',
               'sunflower butter', 'coconut butter', 'cashew butter'},
    'cream': {'coconut cream', 'soy cream', 'oat cream', 'vegan cream', 'cashew cream'},
    'bacon': {'vegan bacon', 'tempeh bacon', 'coconut bacon', 'mushroom bacon'},
    'egg': {'flaxseed', 'chia', 'aquafaba', 'vegan egg', 'plant-based egg', 'apple sauce'}
}

VEGAN_MODIFIERS = {
    'vegan', '100% vegan', 'plant-based', 'dairy-free', 'egg-free',
    'no milk', 'no eggs', 'cruelty-free', 'without animal derivatives',
    '100% plant-based', 'meat-free', 'no animal products'
}

# --- UTILITY (Used in vegan, vegetarian, lactose free and gluten free)
def parse_ingredients(ingredients_input):
    """Always returning a parsed list of ingredients"""
    if isinstance(ingredients_input, list):
        return ingredients_input
    if isinstance(ingredients_input, str) and ingredients_input.strip():
        try:
            return ast.literal_eval(ingredients_input)
        except:
            return [
                x.strip().strip('"').strip("'")
                for x in re.split(r',(?![^[]*\])', ingredients_input.strip('[]'))
                if x.strip()
            ]
    return []

def contains_vegan_exception(ingredient, keyword):
    return any(re.search(rf'\b{re.escape(ex)}\b', ingredient, re.IGNORECASE)
               for ex in VEGAN_EXCEPTIONS.get(keyword, set()))

# --- Classification
def classify_vegan(ingredients_input):
    """Returning True or False based on the ingredients"""
    ingredients = parse_ingredients(ingredients_input)
    for ing in ingredients:
        if not isinstance(ing, str):
            continue
        ing_lower = ing.lower()
        if any(skip in ing_lower for skip in ('water', 'salt')):
            continue
        if any(re.search(rf'\b{re.escape(mod)}\b', ing_lower) for mod in VEGAN_MODIFIERS):
            continue
        for keyword in NON_VEGAN_KEYWORDS:
            if re.search(rf'\b{re.escape(keyword)}(?:s)?\b', ing_lower):
                if not contains_vegan_exception(ing_lower, keyword):
                    return False
    return True

def get_non_vegan_ingredients(ingredients_input):
    """Returning list of ingredients without vegan exceptions"""
    ingredients = parse_ingredients(ingredients_input)
    non_vegan = []
    for ing in ingredients:
        if not isinstance(ing, str):
            continue
        ing_lower = ing.lower()
        if any(skip in ing_lower for skip in ('water', 'salt')):
            continue
        for keyword in NON_VEGAN_KEYWORDS:
            if re.search(rf'\b{re.escape(keyword)}(?:s)?\b', ing_lower):
                if not contains_vegan_exception(ing_lower, keyword):
                    non_vegan.append(ing)
                break
    return non_vegan

# --- Apply
df['VEGAN'] = df['NER_clean'].apply(classify_vegan)
df['NON_VEGAN_INGREDIENTS'] = df['NER_clean'].apply(get_non_vegan_ingredients)

# --- Final validation
anti_patterns = re.compile(
    r'\b(not vegan|meat|steak|fish|cheese|egg|eggs|beef|chicken|pork|bacon|cream)\b',
    re.IGNORECASE
)

df['VEGAN'] = df.apply(
    lambda row: False if row['VEGAN'] and isinstance(row['TITLE'], str) and anti_patterns.search(row['TITLE']) else row['VEGAN'],
    axis=1
)

# View results
df[['TITLE', 'NER', 'VEGAN', 'NON_VEGAN_INGREDIENTS']].head(200)

,TITLE,NER,VEGAN,NON_VEGAN_INGREDIENTS
0,Western Sizzlin Bread Pudding,"['eggs', 'milk', 'sugar', 'vanilla', 'gallon w...",False,"[eggs, milk]"
1,Creamy Tuna And Bacon Salad (Reduced Fat),"['tuna', 'mayonnaise', 'low-fat sour cream', '...",False,"[tuna, mayonnaise, low-fat sour cream, bacon b..."
2,Spinach And Mushroom Lasagna,"['olive oil', 'onion', 'garlic', 'salt', 'grou...",False,"[ricotta cheese, Parmesan cheese, egg, mozzare..."
3,Three-Bean Tacos,"['olive oil', 'onion', 'red bell pepper', 'gre...",False,[cheddar cheese]
4,Hearty Hamburger Soup,"['lean ground beef', 'white onion', 'ground bl...",False,"[lean ground beef, parmesan cheese, sour cream]"
...,...,...,...,...
195,Sweet Potato Brotchen (Bread Rolls),"['sweet potatoes', 'butter', 'salt', 'flour', ...",False,[butter]
196,Green Cleaner and Bug Repellant in One (Concen...,"['essential oil', 'vodka', 'liquid soap']",True,[]
197,Chocolate Chip Pie,"['graham cracker crust', 'marshmallows', 'milk...",False,"[milk, whipping cream]"
198,Poppy Seed Dressing,"['sugar', 'dry mustard', 'salt', 'vinegar', 'o...",True,[]


In [11]:
# List of non vegetarian ingredients
NON_VEGETARIAN_KEYWORDS = {
    'meat', 'red meat', 'white meat', 'poultry', 'veal', 'lamb', 'duck',
    'goose', 'chicken', 'beef', 'pork', 'bacon', 'ham', 'prosciutto',
    'sausage', 'salami', 'hot dog', 'pepperoni', 'mortadella', 'chorizo',
    'ground beef', 'minced meat', 'steak', 'ribs', 'brisket', 'lardons',
    'fish', 'anchovy', 'anchovies', 'tuna', 'salmon', 'sardines', 'mackerel',
    'cod', 'haddock', 'tilapia', 'trout', 'snapper', 'catfish', 'bass',
    'shellfish', 'shrimp', 'prawns', 'crab', 'lobster', 'scampi', 'clam',
    'mussels', 'oysters', 'scallops', 'caviar', 'roe',
    'gelatin', 'lard', 'collagen', 'isinglass', 'pepsin', 'suet', 'bone broth',
    'bone marrow', 'stock (meat)', 'bouillon (meat)', 'chicken broth', 'beef broth'
}

# Exceptions for vegetarian subs
VEGETARIAN_EXCEPTIONS = {
    'meat': {'soy', 'soy protein', 'textured vegetable protein', 'tvp', 'seitan', 'tofu',
             'tempeh', 'jackfruit', 'lentils', 'chickpeas', 'plant-based', 'mycoprotein', 'quorn'},
    'bacon': {'vegan bacon', 'tempeh bacon', 'coconut bacon', 'rice paper bacon'},
    'gelatin': {'agar', 'pectin', 'carrageenan', 'guar gum', 'xanthan gum'},
    'fish': {'banana blossom', 'tofu', 'plant-based', 'seaweed', 'nori', 'smoked carrot'}
}

VEGETARIAN_MODIFIERS = {
    'vegetarian', 'veggie', 'plant-based', 'meatless', 'no meat',
    'without meat', 'cruelty-free', 'ovo-vegetarian', 'lacto-vegetarian',
    '100% vegetarian', 'meat-free', 'green option'
}

def contains_vegetarian_exception(ingredient, keyword):
    exceptions = VEGETARIAN_EXCEPTIONS.get(keyword, set())
    return any(re.search(rf'\b{re.escape(ex)}\b', ingredient, re.IGNORECASE) for ex in exceptions)

# --- Classification
def classify_vegetarian(ingredients_input):
    """Returning True or False based on the ingredients"""
    ingredients = parse_ingredients(ingredients_input) # using the function 'parse_ingredients' defined in the previous cell
    for ing in ingredients:
        if not isinstance(ing, str):
            continue
        ing_lower = ing.lower()
        if any(skip in ing_lower for skip in ('water', 'salt')):
            continue
        if any(re.search(rf'\b{re.escape(mod)}\b', ing_lower) for mod in VEGETARIAN_MODIFIERS):
            continue
        for keyword in NON_VEGETARIAN_KEYWORDS:
            if re.search(rf'\b{re.escape(keyword)}(?:s)?\b', ing_lower):
                if not contains_vegetarian_exception(ing_lower, keyword):
                    return False
    return True

def get_non_vegetarian_ingredients(ingredients_input):
    """Returning list of ingredients without vegetarian exceptions"""
    ingredients = parse_ingredients(ingredients_input)
    non_vegetarian = []
    for ing in ingredients:
        if not isinstance(ing, str):
            continue
        ing_lower = ing.lower()
        if any(skip in ing_lower for skip in ('water', 'salt')):
            continue
        for keyword in NON_VEGETARIAN_KEYWORDS:
            if re.search(rf'\b{re.escape(keyword)}(?:s)?\b', ing_lower):
                if not contains_vegetarian_exception(ing_lower, keyword):
                    non_vegetarian.append(ing)
                break
    return non_vegetarian

# Apply to dataframe
df['VEGETARIAN'] = df['NER_clean'].apply(classify_vegetarian)
df['NON_VEGETARIAN_INGREDIENTS'] = df['NER_clean'].apply(get_non_vegetarian_ingredients)

anti_patterns = re.compile(
    r'\b(not vegetarian|not veg|meat|steak|fish|beef|chicken|pork|bacon)\b',
    re.IGNORECASE
)

df['VEGETARIAN'] = df.apply(
    lambda row: False if row['VEGETARIAN'] and isinstance(row['TITLE'], str) and anti_patterns.search(row['TITLE']) else row['VEGETARIAN'],
    axis=1
)

# Show results
df[['TITLE', 'NER', 'VEGETARIAN', 'NON_VEGETARIAN_INGREDIENTS']].head(200)

,TITLE,NER,VEGETARIAN,NON_VEGETARIAN_INGREDIENTS
0,Western Sizzlin Bread Pudding,"['eggs', 'milk', 'sugar', 'vanilla', 'gallon w...",True,[]
1,Creamy Tuna And Bacon Salad (Reduced Fat),"['tuna', 'mayonnaise', 'low-fat sour cream', '...",False,"[tuna, bacon bits]"
2,Spinach And Mushroom Lasagna,"['olive oil', 'onion', 'garlic', 'salt', 'grou...",True,[]
3,Three-Bean Tacos,"['olive oil', 'onion', 'red bell pepper', 'gre...",True,[]
4,Hearty Hamburger Soup,"['lean ground beef', 'white onion', 'ground bl...",False,[lean ground beef]
...,...,...,...,...
195,Sweet Potato Brotchen (Bread Rolls),"['sweet potatoes', 'butter', 'salt', 'flour', ...",True,[]
196,Green Cleaner and Bug Repellant in One (Concen...,"['essential oil', 'vodka', 'liquid soap']",True,[]
197,Chocolate Chip Pie,"['graham cracker crust', 'marshmallows', 'milk...",True,[]
198,Poppy Seed Dressing,"['sugar', 'dry mustard', 'salt', 'vinegar', 'o...",True,[]


## Definizione del metodo di preparazione della ricetta

In [8]:
# Extended map for methods and keywords
cooking_methods = {
    'Baked': [
        'bake', 'baked', 'baking', 'preheat oven', 'pre-heated oven', 'ovenproof',
        'place in oven', 'roast in oven', 'bake at', 'oven-bake', 'put in oven', 'oven cook'
    ],
    'Boiled': [
        'boil', 'boiled', 'boiling', 'bring to a boil', 'simmer', 'parboil', 'blanch',
        'blanched', 'boiled in', 'simmered', 'cook in boiling water', 'gentle boil', 'rapid boil'
    ],
    'Broil': [
        'broil', 'broiled', 'broiling', 'under the broiler', 'place under broiler', 'broiler setting'
    ],
    'Fried': [
        'fry', 'fried', 'frying', 'deep fry', 'deep-fry', 'pan fry', 'pan-fried', 'shallow fry',
        'stir-fry', 'sauté', 'sautée', 'sautéed', 'sauted', 'sautéing', 'sear', 'seared',
        'brown in a pan', 'browned', 'cook in oil', 'crisp in pan', 'fry in skillet', 'flash fry'
    ],
    'Grilled': [
        'grill', 'grilled', 'grilling', 'grill pan', 'on the grill', 'barbecue', 'bbq',
        'cook over grill', 'char-grill', 'chargrilled', 'griddle pan', 'grilled over flame'
    ],
    'Roasted': [
        'roast', 'roasted', 'roasting', 'oven-roasted', 'slow roast', 'roast in oven',
        'roast at', 'roasted until golden', 'dry-roast', 'dry roast'
    ],
    'Slow Cooking': [
        'slow cook', 'slow-cooked', 'slow-cook', 'slow cooking', 'crockpot', 'crock pot',
        'slow cooker', 'cook on low for', 'long cook time', 'set to low', 'set to high and cook for hours'
    ],
    'Steamed': [
        'steam', 'steamed', 'steaming', 'in steamer', 'steamer basket', 'double boiler',
        'steam until tender', 'cook with steam', 'steam over water', 'steam for minutes'
    ],
    'Pressure Cooked': [
        'pressure cook', 'pressure cooked', 'pressure cooker', 'instant pot',
        'cooked under pressure', 'cook under pressure', 'high pressure', 'manual pressure cook'
    ],
    'Microwaved': [
        'microwave', 'microwaved', 'microwaving', 'cook in microwave', 'microwave on high',
        'microwave-safe bowl', 'microwave for minutes'
    ],
    'Sous Vide': [
        'sous vide', 'sous-vide', 'precision cooker', 'immersion circulator',
        'vacuum sealed and cooked in water bath', 'vacuum seal and cook at temperature',
        'cook sous vide'
    ],
    'Toasted': [
        'toast', 'toasted', 'toasting', 'in toaster', 'toasted until golden',
        'under grill until golden', 'toast under broiler', 'toast on skillet'
    ],
    'Blended': [
        'blend', 'blended', 'blending', 'blender', 'puree', 'pureed', 'pureeing',
        'food processor', 'process until smooth', 'blend until creamy', 'blend until combined',
        'whizz in blender', 'pulse until smooth'
    ],
    'Raw': [
        'serve raw', 'uncooked', 'no cook', 'eat raw', 'fresh only', 'as is', 'ready to eat',
        'no heating required', 'chilled and served', 'consume without cooking', 'fresh and raw'
    ]
}

def detect_cooking_method(instruction_list):
    '''A method to detect cooking method'''
    if isinstance(instruction_list, str):
        try:
            steps = ast.literal_eval(instruction_list.lower())
        except:
            steps = [instruction_list.lower()]
    else:
        steps = [str(instruction).lower() for instruction in instruction_list]

    for method, keywords in cooking_methods.items():
        if any(any(kw in step for kw in keywords) for step in steps):
            return method
    return 'Other'

df['COOKING_METHOD'] = df['DIRECTIONS'].apply(detect_cooking_method)

df[['TITLE', 'DIRECTIONS', 'COOKING_METHOD']].head(200)

,TITLE,DIRECTIONS,COOKING_METHOD
0,Western Sizzlin Bread Pudding,"[""Distribute rolls and cinnamon roll in (4) 2\...",Baked
1,Creamy Tuna And Bacon Salad (Reduced Fat),"[""Put everything in a bowl and mix together un...",Other
2,Spinach And Mushroom Lasagna,"[""Heat oven to 375 degrees."", """", ""In medium s...",Baked
3,Three-Bean Tacos,"[""Heat oil in a large skillet over medium-high...",Boiled
4,Hearty Hamburger Soup,"[""Brown ground beef and onion in a large pot. ...",Boiled
...,...,...,...
195,Sweet Potato Brotchen (Bread Rolls),"[""Peel and cut sweet potatoes into chunks."", ""...",Baked
196,Green Cleaner and Bug Repellant in One (Concen...,"[""Mix all ingredients."", ""Store in a glass jar...",Other
197,Chocolate Chip Pie,"[""Make crust (I buy the prepared ones from the...",Boiled
198,Poppy Seed Dressing,"[""Combine first 6 ingredients in a blender con...",Blended


## Euristica in grado di stimare la difficoltà di una ricetta

In [9]:
import ast

def estimate_difficulty(row):
    try:
        ingredients = ast.literal_eval(row['INGREDIENTS'])
        instructions = ast.literal_eval(row['DIRECTIONS'])
    except Exception:
        return 'Unknown'

    num_ingredients = len(ingredients)
    num_steps = len([s for s in instructions if s.strip()])
    instruction_text = ' '.join(instructions).lower()

    mild_keywords = [
        'mix', 'combine', 'stir', 'spread', 'pour', 'season',
        'slice', 'chop', 'cut', 'heat', 'grease', 'add'
    ]
    medium_keywords = [
        'simmer', 'boil', 'bake', 'grill', 'saute', 'blend',
        'fry', 'knead', 'rest', 'whisk', 'preheat', 'chill',
        'melt', 'marinate', 'brown'
    ]
    hard_keywords = [
        'sous vide', 'caramelize', 'double boiler', 'bain-marie',
        'julienne', 'poach', 'deglaze', 'layer', 'water bath',
        'refrigerate overnight', 'fold', 'temper', 'pass through sieve',
        'glaze', 'rotate pans', 'cool completely', 'remove from mold'
    ]

    def keyword_score(keywords, text, max_points=3):
        score = 0
        for kw in keywords:
            matches = len(re.findall(rf'\b{re.escape(kw)}\b', text))
            score += matches
        return min(score, max_points)

    mild_score = keyword_score(mild_keywords, instruction_text)
    medium_score = keyword_score(medium_keywords, instruction_text)
    hard_score = keyword_score(hard_keywords, instruction_text, max_points=4)

    bonus = 0
    if 'overnight' in instruction_text or 'refrigerate overnight' in instruction_text:
        bonus += 3
    if 'cool completely' in instruction_text or 'let cool' in instruction_text:
        bonus += 2
    if 'food processor' in instruction_text:
        bonus += 1
    if 'strain' in instruction_text or 'sieve' in instruction_text:
        bonus += 1
    if 'double boiler' in instruction_text or 'bain-marie' in instruction_text:
        bonus += 2
    if 'fold' in instruction_text:
        bonus += 1

    complex_ingredients = ['gelatin', 'saffron', 'truffle', 'wasabi', 'caviar']
    complex_ingredients_count = sum(1 for ingr in ingredients if any(ci in ingr.lower() for ci in complex_ingredients))
    bonus += complex_ingredients_count * 1.5

    score = 0
    score += num_ingredients * 0.25
    score += num_steps * 1.5
    score += mild_score * 1
    score += medium_score * 2
    score += hard_score * 3
    score += bonus

    # thresholds for the 3 classes (da rivedere)
    if score <= 10:
        return 'Easy'
    elif score <= 22:
        return 'Medium'
    else:
        return 'Hard'

# Apply
df['DIFFICULTY'] = df.apply(estimate_difficulty, axis=1)

df[['TITLE', 'DIRECTIONS', 'DIFFICULTY']].head(200)

,TITLE,DIRECTIONS,DIFFICULTY
0,Western Sizzlin Bread Pudding,"[""Distribute rolls and cinnamon roll in (4) 2\...",Medium
1,Creamy Tuna And Bacon Salad (Reduced Fat),"[""Put everything in a bowl and mix together un...",Easy
2,Spinach And Mushroom Lasagna,"[""Heat oven to 375 degrees."", """", ""In medium s...",Hard
3,Three-Bean Tacos,"[""Heat oil in a large skillet over medium-high...",Hard
4,Hearty Hamburger Soup,"[""Brown ground beef and onion in a large pot. ...",Hard
...,...,...,...
195,Sweet Potato Brotchen (Bread Rolls),"[""Peel and cut sweet potatoes into chunks."", ""...",Hard
196,Green Cleaner and Bug Repellant in One (Concen...,"[""Mix all ingredients."", ""Store in a glass jar...",Easy
197,Chocolate Chip Pie,"[""Make crust (I buy the prepared ones from the...",Hard
198,Poppy Seed Dressing,"[""Combine first 6 ingredients in a blender con...",Medium


## Le celle sottostanti contengono una logica in grado di determinare se le ricette contengono oppure no lattosio e glutine

In [10]:
# List of lactose ingredients
LACTOSE_KEYWORDS = {
    # milks and derivatives
    'milk', 'whole milk', 'skim milk', 'raw milk', 'lactose', 'lactose monohydrate',
    'cream', 'heavy cream', 'double cream', 'single cream', 'whipping cream',
    'butter', 'ghee', 'clarified butter',

    # cheese
    'cheese', 'cream cheese', 'soft cheese', 'hard cheese',
    'ricotta', 'mozzarella', 'parmesan', 'pecorino', 'feta', 'brie', 'camembert',
    'blue cheese', 'goat cheese', 'cheddar', 'colby', 'gouda', 'gruyere',
    'swiss cheese', 'romano cheese', 'mascarpone', 'paneer', 'provolone',

    # Yogurt and similar
    'yogurt', 'greek yogurt', 'kefir', 'curd',

    # desserts and industrials
    'custard', 'ice cream', 'gelato', 'milkshake', 'pudding', 'white chocolate',

    # technical ingredients
    'whey', 'whey protein', 'whey powder', 'casein', 'caseinate', 'milk solids',
    'milk powder', 'dried milk', 'nonfat milk', 'low-fat milk', 'milk protein',
    'milkfat', 'buttermilk', 'evaporated milk', 'condensed milk', 'recombined milk'
}

# Exceptions for lactose free variants
LACTOSE_EXCEPTIONS = {
    'milk': {
        'almond milk', 'soy milk', 'oat milk', 'coconut milk', 'rice milk',
        'lactose-free milk', 'hemp milk', 'pea milk', 'cashew milk',
        'hazelnut milk', 'macadamia milk', 'flax milk'
    },
    'cheese': {
        'vegan cheese', 'plant-based cheese', 'nut cheese', 'cashew cheese',
        'almond cheese', 'tofu cheese', 'soy cheese'
    },
    'butter': {
        'vegan butter', 'plant-based butter', 'margarine', 'oil spread'
    },
    'cream': {
        'coconut cream', 'soy cream', 'oat cream', 'cashew cream', 'almond cream'
    },
    'yogurt': {
        'coconut yogurt', 'soy yogurt', 'almond yogurt', 'oat yogurt',
        'lactose-free yogurt', 'rice yogurt', 'cashew yogurt'
    }
}

def contains_lactose_exception(ingredient, keyword):
    exceptions = LACTOSE_EXCEPTIONS.get(keyword, set())
    return any(re.search(rf'\b{re.escape(ex)}\b', ingredient, re.IGNORECASE) for ex in exceptions)

def classify_lactose_free(ingredients_input):
    ingredients = parse_ingredients(ingredients_input)
    for ing in ingredients:
        if not isinstance(ing, str):
            continue
        ing_lower = ing.lower()
        for keyword in LACTOSE_KEYWORDS:
            if re.search(rf'\b{re.escape(keyword)}(?:s)?\b', ing_lower):
                if not contains_lactose_exception(ing_lower, keyword):
                    return False
    return True

def get_lactose_ingredients(ingredients_input):
    ingredients = parse_ingredients(ingredients_input)
    lactose = []
    for ing in ingredients:
        if not isinstance(ing, str):
            continue
        ing_lower = ing.lower()
        for keyword in LACTOSE_KEYWORDS:
            if re.search(rf'\b{re.escape(keyword)}(?:s)?\b', ing_lower):
                if not contains_lactose_exception(ing_lower, keyword):
                    lactose.append(ing)
                break
    return lactose

# Apply to dataframe
df['LACTOSE_FREE'] = df['NER_clean'].apply(classify_lactose_free)
df['LACTOSE_INGREDIENTS'] = df['NER_clean'].apply(get_lactose_ingredients)

# Show results
df[['TITLE', 'NER', 'LACTOSE_FREE', 'LACTOSE_INGREDIENTS']].head(200)

,TITLE,NER,LACTOSE_FREE,LACTOSE_INGREDIENTS
0,Western Sizzlin Bread Pudding,"['eggs', 'milk', 'sugar', 'vanilla', 'gallon w...",False,[milk]
1,Creamy Tuna And Bacon Salad (Reduced Fat),"['tuna', 'mayonnaise', 'low-fat sour cream', '...",False,[low-fat sour cream]
2,Spinach And Mushroom Lasagna,"['olive oil', 'onion', 'garlic', 'salt', 'grou...",False,"[ricotta cheese, Parmesan cheese, mozzarella]"
3,Three-Bean Tacos,"['olive oil', 'onion', 'red bell pepper', 'gre...",False,[cheddar cheese]
4,Hearty Hamburger Soup,"['lean ground beef', 'white onion', 'ground bl...",False,"[parmesan cheese, sour cream]"
...,...,...,...,...
195,Sweet Potato Brotchen (Bread Rolls),"['sweet potatoes', 'butter', 'salt', 'flour', ...",False,[butter]
196,Green Cleaner and Bug Repellant in One (Concen...,"['essential oil', 'vodka', 'liquid soap']",True,[]
197,Chocolate Chip Pie,"['graham cracker crust', 'marshmallows', 'milk...",False,"[milk, whipping cream]"
198,Poppy Seed Dressing,"['sugar', 'dry mustard', 'salt', 'vinegar', 'o...",True,[]


In [11]:
# List of gluten ingredients
GLUTEN_KEYWORDS = {
    # cereals containing gluten
    'wheat', 'barley', 'rye', 'spelt', 'kamut', 'triticale', 'farro', 'einkorn', 'emmer',

    # derivatives
    'semolina', 'durum', 'graham flour', 'matzo', 'gluten', 'vital wheat gluten',
    'hydrolyzed wheat protein', 'wheat starch', 'modified wheat starch',
    'malted barley', 'malted milk', 'malt', 'malt extract', 'malt syrup', 'malt vinegar',

    # industrial ingredients
    'brewer’s yeast', 'brewer yeast', 'yeast extract', 'textured vegetable protein (wheat)',
    'vegetable starch (wheat)', 'vegetable gum (wheat)', 'natural flavor (wheat)',
    'seasoned flour', 'spice mix (with wheat)',

    # common products
    'seitan', 'bran', 'bread', 'pasta', 'noodles', 'crackers', 'breadcrumbs', 'biscuit',
    'cake flour', 'pastry flour', 'self-rising flour', 'bagel', 'croissant', 'brioche',
    'pancake mix', 'waffle mix', 'muffin mix', 'pie crust', 'pizza crust', 'tortilla (wheat)',

    # transformed products
    'beer', 'ale', 'lager', 'stuffing mix', 'gravy mix', 'roux', 'soups (with pasta/noodles)',
    'sauces (with flour)', 'soy sauce', 'teriyaki sauce', 'tempura', 'coating mix', 'breading',

    # desserts and snacks
    'ladyfinger', 'graham cracker', 'pretzel', 'cereal bar', 'energy bar (with gluten grains)'
}

# Exceptions for gluten free
GLUTEN_EXCEPTIONS = {
    'wheat': {'wheatgrass', 'wheat-free tamari'},
    'malt': {'maltodextrin (from corn)', 'maltodextrin from corn', 'rice malt', 'corn malt'},
    'starch': {
        'corn starch', 'potato starch', 'tapioca starch', 'arrowroot starch',
        'sago starch', 'rice starch'
    },
    'flour': {
        'almond flour', 'coconut flour', 'chickpea flour', 'rice flour', 'corn flour',
        'potato flour', 'soy flour', 'buckwheat flour', 'tapioca flour', 'quinoa flour',
        'sorghum flour', 'teff flour', 'millet flour', 'amaranth flour', 'cassava flour',
        'oat flour (gluten-free)', 'oat flour gluten-free', 'lentil flour', 'pea flour'
    },
    'bran': {
        'rice bran', 'corn bran', 'oat bran (gluten-free)', 'oat bran gluten-free'
    },
    'bread': {'gluten-free bread'},
    'pasta': {'gluten-free pasta', 'rice pasta', 'quinoa pasta'},
    'noodles': {'rice noodles', 'glass noodles', 'sweet potato noodles', 'mung bean noodles'},
    'beer': {'gluten-free beer', 'sorghum beer'},
    'cracker': {'gluten-free crackers'},
    'breadcrumbs': {'gluten-free breadcrumbs', 'crushed cornflakes (GF)'},
    'biscuit': {'gluten-free biscuit'},
    'soy sauce': {'tamari (gluten-free)', 'gluten-free soy sauce'},
    'gravy mix': {'gluten-free gravy'},
    'stuffing mix': {'gluten-free stuffing'},
    'pizza crust': {'gluten-free pizza crust'},
    'cereal bar': {'gluten-free cereal bar'},
    'breading': {'gluten-free breading'},
    'coating mix': {'gluten-free coating'}
}

def contains_gluten_exception(ingredient, keyword):
    exceptions = GLUTEN_EXCEPTIONS.get(keyword, set())
    return any(re.search(rf'\b{re.escape(ex)}\b', ingredient, re.IGNORECASE) for ex in exceptions)

def classify_gluten_free(ingredients_input):
    ingredients = parse_ingredients(ingredients_input)
    for ing in ingredients:
        if not isinstance(ing, str):
            continue
        ing_lower = ing.lower()
        for keyword in GLUTEN_KEYWORDS:
            if re.search(rf'\b{re.escape(keyword)}(?:s)?\b', ing_lower):
                if not contains_gluten_exception(ing_lower, keyword):
                    return False
    return True

def get_gluten_ingredients(ingredients_input):
    ingredients = parse_ingredients(ingredients_input)
    gluten = []
    for ing in ingredients:
        if not isinstance(ing, str):
            continue
        ing_lower = ing.lower()
        for keyword in GLUTEN_KEYWORDS:
            if re.search(rf'\b{re.escape(keyword)}(?:s)?\b', ing_lower):
                if not contains_gluten_exception(ing_lower, keyword):
                    gluten.append(ing)
                break
    return gluten

# Apply to dataframe
df['GLUTEN_FREE'] = df['NER_clean'].apply(classify_gluten_free)
df['GLUTEN_INGREDIENTS'] = df['NER_clean'].apply(get_gluten_ingredients)

# Show results
df[['TITLE', 'NER', 'GLUTEN_FREE', 'GLUTEN_INGREDIENTS']].head(200)

,TITLE,NER,GLUTEN_FREE,GLUTEN_INGREDIENTS
0,Western Sizzlin Bread Pudding,"['eggs', 'milk', 'sugar', 'vanilla', 'gallon w...",True,[]
1,Creamy Tuna And Bacon Salad (Reduced Fat),"['tuna', 'mayonnaise', 'low-fat sour cream', '...",True,[]
2,Spinach And Mushroom Lasagna,"['olive oil', 'onion', 'garlic', 'salt', 'grou...",False,[lasagna noodles]
3,Three-Bean Tacos,"['olive oil', 'onion', 'red bell pepper', 'gre...",True,[]
4,Hearty Hamburger Soup,"['lean ground beef', 'white onion', 'ground bl...",False,[soy sauce]
...,...,...,...,...
195,Sweet Potato Brotchen (Bread Rolls),"['sweet potatoes', 'butter', 'salt', 'flour', ...",False,[bread flour]
196,Green Cleaner and Bug Repellant in One (Concen...,"['essential oil', 'vodka', 'liquid soap']",True,[]
197,Chocolate Chip Pie,"['graham cracker crust', 'marshmallows', 'milk...",False,[graham cracker crust]
198,Poppy Seed Dressing,"['sugar', 'dry mustard', 'salt', 'vinegar', 'o...",True,[]


## La cella sottostante può essere eseguita solo su computer potenti, e il suo scopo principale è quello di generare la categoria di prezzo della ricetta, e usa un diverso approccio anche per definire altre categorie a cui appartengono le ricette

Qui inizia la magia:

In [19]:
import numpy as np
import torch
from transformers import pipeline
import re
from tqdm import tqdm

gpu = torch.cuda.get_device_name(0)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device count: {torch.cuda.device_count()}")
print(f"Current device: {torch.cuda.current_device()}")
print(f"Device name: {gpu}")

# Eseguire questa cella solo se runnata dal Barone
if gpu == "NVIDIA GeForce RTX 3060 Ti":
    classifier = pipeline(
        task="zero-shot-classification",
        model="MoritzLaurer/deberta-v3-base-zeroshot-v1.1-all-33",
        device=0,
        torch_dtype=torch.float16,
        model_kwargs={"cache_dir": "./cache"},
        batch_size=32,  # Optimal for RTX 3060 Ti
        framework="pt"
    )
    
    # %% Data Loading
    df['NER'] = df['NER'].apply(eval)  # Convert string lists to actual lists
    
    # %% Food Categories & Prices (EUR/kg)
    CATEGORIES = [
        "dairy", "meat", "seafood", "grain", "vegetable",
        "fruit", "spice/herb", "processed", "sweetener",
        "condiment", "legume", "oil/fat"
    ]
    
    MEDIAN_PRICES = {
        "dairy": 3.50,       # Milk, cheese
        "meat": 7.50,        # Chicken, beef
        "seafood": 12.00,    # Fish, shrimp
        "grain": 2.20,       # Flour, rice
        "vegetable": 1.80,   # Onions, garlic
        "fruit": 2.50,       # Tomatoes, bananas
        "spice/herb": 18.00, # Vanilla, cinnamon
        "processed": 4.50,   # Pasta, canned goods
        "sweetener": 2.20,   # Sugar
        "condiment": 5.00,   # Mayo, dressings
        "legume": 3.00,      # Beans, lentils
        "oil/fat": 8.00      # Olive oil
    }
    
    # %% Ingredient Cleaning (Fixed)
    def clean_ingredient(ingredient: str) -> str:
        """Conservative cleaning preserving ingredient names"""
        # Remove quantities (e.g., "200g", "1/2 cup")
        cleaned = re.sub(r'\b\d+[\d/\.]*\s*[a-z]*\b', '', ingredient, flags=re.IGNORECASE)
        # Remove special chars except spaces
        cleaned = re.sub(r'[^\w\s]', '', cleaned).strip().lower()
        return cleaned if cleaned else "unknown"
    
    # %% Batch Classification (GPU-optimized)
    classification_cache = {}
    
    def batch_classify(ingredients: list, batch_size: int = 16) -> dict:  # Reduced batch size
        unique_ingredients = list(set(ingredients))
    
        with torch.no_grad():  # Disable gradient tracking
            for batch in tqdm([unique_ingredients[i:i+batch_size]
                               for i in range(0, len(unique_ingredients), batch_size)],
                              desc="Classifying Ingredients"):
                # Process batch on GPU
                results = classifier(batch, CATEGORIES, multi_label=False)
    
                # Cache results
                for ing, result in zip(batch, results):
                    classification_cache[ing] = result['labels'][0]
    
                # Clear GPU cache
                torch.cuda.empty_cache()
    
        return classification_cache
    
    # %% Process Entire Dataset
    # Get all unique ingredients
    all_ingredients = [clean_ingredient(ing)
                       for recipe in df['NER']
                       for ing in recipe]
    unique_ingredients = list(set(all_ingredients))
    
    # Batch classify with progress bar
    _ = batch_classify(unique_ingredients, batch_size=64)
    
    # Calculate recipe costs
    df['total_cost'] = df['NER'].apply(
        lambda x: round(sum(
            MEDIAN_PRICES.get(classification_cache[clean_ingredient(ing)], 3.00)
            for ing in x
        ), 2)
    )
    
    # Dynamic price categorization
    costs = df['total_cost'].values
    
    # threshold per le categorie degli prezzi
    very_cheap, cheap, medium, expensive = np.percentile(costs, [20, 40, 60, 85])
    
    df['price_tag'] = df['total_cost'].apply(
        lambda x: 'very cheap' if x <= very_cheap
        else 'cheap' if x <= cheap
        else 'medium' if x <= medium
        else 'expensive' if x <= expensive
        else 'rich'
    )

    def get_categories(ingredient_list):
        return [classification_cache.get(ing) for ing in ingredient_list]
    
    def is_vegan(cat_list):
        return all(cat not in ['dairy', 'meat', 'seafood'] for cat in cat_list if cat is not None)
    
    def is_vegetarian(cat_list):
        return all(cat not in ['meat', 'seafood'] for cat in cat_list if cat is not None)
    
    df['categories'] = df['NER_clean'].apply(get_categories)
    df['vegan'] = df['categories'].apply(is_vegan)
    df['vegetarian'] = df['categories'].apply(is_vegetarian)
    
    # Save outputs
    df.to_csv('full_tagged_dataset_10%', index=False)

C:\Users\aless\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AssertionError: Torch not compiled with CUDA enabled

Qui la magia finisce :)

# Creazione del modello per la classificazione del tag 'PREPARATION_TIME' e testing

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Default parameters
MAX_LEN = 500
NUM_WORDS = 20000
BATCH_SIZE = 32
EPOCHS = 10

# Method to preprocess a dataframe
def preprocess(df, text_columns, label_column, n=10000):
    df = df[text_columns + [label_column]].dropna().head(n).copy()
    df[text_columns[0]] = df[text_columns[0]].apply(eval)
    df[text_columns[1]] = df[text_columns[1]].apply(eval)
    df["full_text"] = df[text_columns[0]].apply(lambda x: " ".join(x)) + " " + df[text_columns[1]].apply(lambda x: " ".join(x))

    label_encoder = LabelEncoder()
    df["encoded_label"] = label_encoder.fit_transform(df[label_column])

    return df, label_encoder

# Method to train a classificator
def train_text_classifier(df, label_encoder, model_name_prefix):
    X = df["full_text"]
    y = df["encoded_label"]

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    tokenizer = Tokenizer(num_words=NUM_WORDS, oov_token="<OOV>")
    tokenizer.fit_on_texts(X_train)

    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_val_seq = tokenizer.texts_to_sequences(X_val)

    X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding="post", truncating="post")
    X_val_pad = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding="post", truncating="post")

    model = Sequential([
        Embedding(input_dim=NUM_WORDS, output_dim=128, input_length=MAX_LEN),
        GlobalMaxPooling1D(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(len(label_encoder.classes_), activation='softmax')
    ])

    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(
        X_train_pad, y_train,
        validation_data=(X_val_pad, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[early_stop]
    )

    # Saving model
    model.save(f"classifiers/prep_time/{model_name_prefix}.h5")
    with open(f"classifiers/prep_time/{model_name_prefix}_label_mapping.pkl", "wb") as f:
        pickle.dump(dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))), f)

    with open(f"classifiers/prep_time/{model_name_prefix}_tokenizer.pkl", "wb") as f:
        pickle.dump(tokenizer, f)

    print(f"Modello '{model_name_prefix}' salvato con successo.")
    return model

# === FIRST MODEL: PREPARATION_TIME ===
df_time = df[["INGREDIENTS", "DIRECTIONS", "PREPARATION_TIME"]].copy()
df_time, prep_time_encoder = preprocess(df_time, ["INGREDIENTS", "DIRECTIONS"], "PREPARATION_TIME")
prep_time_model = train_text_classifier(df_time, prep_time_encoder, model_name_prefix="prep_time_classifier")

C:\Users\aless\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - accuracy: 0.3440 - loss: 1.5039 - val_accuracy: 0.5595 - val_loss: 1.0874
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.5825 - loss: 1.0628 - val_accuracy: 0.6415 - val_loss: 0.9110
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 34ms/step - accuracy: 0.6505 - loss: 0.9110 - val_accuracy: 0.6780 - val_loss: 0.8400
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 34ms/step - accuracy: 0.6937 - loss: 0.8323 - val_accuracy: 0.6880 - val_loss: 0.8132
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.7184 - loss: 0.7549 - val_accuracy: 0.6755 - val_loss: 0.8112
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 30ms/step - accuracy: 0.7518 - loss: 0.6918 - val_accuracy: 0.6895 - val_loss: 0.8040
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy: 0.7634 - loss: 0.6626 - val_accuracy: 0.6875 - val_loss: 0.8012
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.7805 - loss: 0.6040 - val_acc

Modello 'prep_time_classifier' salvato con successo.


In [16]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

MAX_LEN = 500  # same as training

# --- Load model and tokenizer ---
prep_time_model = load_model("classifiers/prep_time/prep_time_classifier.h5")

with open("classifiers/prep_time/prep_time_classifier_tokenizer.pkl", "rb") as f:
    prep_time_tokenizer = pickle.load(f)

with open("classifiers/prep_time/prep_time_classifier_label_mapping.pkl", "rb") as f:
    prep_time_label_map = pickle.load(f)

# Reverse dictionaries to decode predictions
inv_prep_time_label_map = {v: k for k, v in prep_time_label_map.items()}

# List of 10 recipes with expected time
recipes = [
    {
        "ingredients": "200g spaghetti, 2 cloves garlic, chili pepper, extra virgin olive oil, salt",
        "instructions": "Cook the spaghetti. Meanwhile, sauté garlic and chili in the oil. Mix with the pasta.",
        "expected_time": "15 min"
    },
    {
        "ingredients": "Lettuce, 1 can of tuna, tomatoes, black olives, onion, olive oil, salt",
        "instructions": "Wash and cut the vegetables. Add the drained tuna and olives. Dress with oil and salt.",
        "expected_time": "10 min"
    },
    {
        "ingredients": "300g Carnaroli rice, 1 saffron sachet, vegetable broth, butter, onion, Parmesan cheese",
        "instructions": "Sauté the onion, add rice and deglaze with wine. Gradually pour in the broth. Add saffron and stir in butter and cheese.",
        "expected_time": "30 min"
    },
    {
        "ingredients": "3 eggs, salt, pepper, butter",
        "instructions": "Beat the eggs with salt and pepper. Pour into a pan with melted butter and stir until set.",
        "expected_time": "5 min"
    },
    {
        "ingredients": "Lasagna sheets, meat sauce, béchamel, Parmesan cheese",
        "instructions": "Layer pasta, sauce, béchamel and cheese. Bake at 180°C for 40 minutes.",
        "expected_time": "60 min"
    },
    {
        "ingredients": "Apples, bananas, oranges, strawberries, lemon juice, sugar",
        "instructions": "Cut the fruit into pieces, mix with lemon juice and sugar, refrigerate.",
        "expected_time": "10 min"
    },
    {
        "ingredients": "Chicken breast, onion, curry powder, cream or coconut milk, oil, salt",
        "instructions": "Brown the onion, add diced chicken, curry, and cream. Cook until the chicken is tender.",
        "expected_time": "25 min"
    },
    {
        "ingredients": "Ladyfingers, mascarpone, eggs, sugar, coffee, cocoa powder",
        "instructions": "Make a cream with yolks, sugar and mascarpone. Layer ladyfingers soaked in coffee and the cream. Dust with cocoa and chill.",
        "expected_time": "30 min"
    },
    {
        "ingredients": "2 eggs, 250ml milk, 125g flour, sugar, butter",
        "instructions": "Mix flour, eggs and milk into a batter. Cook in pan with butter. Fill as desired.",
        "expected_time": "20 min"
    },
    {
        "ingredients": "Mixed dried legumes, onion, carrot, celery, vegetable broth, oil, salt",
        "instructions": "Soak legumes for 12 hours. Sauté onion, carrot, celery. Add legumes and broth and cook for at least an hour.",
        "expected_time": "90 min"
    }
]

# Prepare input texts for models (ingredients + instructions)
texts = [r["ingredients"] + " " + r["instructions"] for r in recipes]

# Tokenize and padding
prep_sequences = prep_time_tokenizer.texts_to_sequences(texts)
prep_padded = pad_sequences(prep_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

# Predicting time with probabilities
prep_preds = prep_time_model.predict(prep_padded)
prep_classes = prep_preds.argmax(axis=1)
prep_labels = [inv_prep_time_label_map[c] for c in prep_classes]

# Print results with probabilities
for i, recipe in enumerate(recipes):
    print(f"Recipe {i+1}:")
    print(f"  Expected preparation time: {recipe['expected_time']} minutes")
    print(f"  Predicted preparation time: {prep_labels[i]}")
    prep_probs_str = ", ".join([f"{inv_prep_time_label_map[j]}: {prep_preds[i][j]*100:.2f}%" for j in range(len(prep_preds[i]))])
    print(f"  Preparation time probabilities: {prep_probs_str}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Recipe 1:
  Expected preparation time: 15 min minutes
  Predicted preparation time: Fast (10-20 mins)
  Preparation time probabilities: Fast (10-20 mins): 47.13%, Medium (20-40 mins): 32.66%, Slow (40-90 mins): 5.50%, Very fast (0-10 mins): 12.92%, Very slow (90+ mins): 1.79%
Recipe 2:
  Expected preparation time: 10 min minutes
  Predicted preparation time: Fast (10-20 mins)
  Preparation time probabilities: Fast (10-20 mins): 68.13%, Medium (20-40 mins): 23.42%, Slow (40-90 mins): 1.01%, Very fast (0-10 mins): 7.07%, Very slow (90+ mins): 0.36%
Recipe 3:
  Expected preparation time: 30 min minutes
  Predicted preparation time: Medium (20-40 mins)
  Preparation time probabilities: Fast (10-20 mins): 13.88%, Medium (20-40 mins): 62.13%, Slow (40-90 mins): 14.24%, Very fast (0-10 mins): 7.64%, Very slow (90+ mins): 2.11%
Recipe 4:
  Expected preparation time: 5 min minutes
  Predicted preparation time: Medium (20-40 mins)
  Preparation time probabil

# Creazione del modello per la classificazione del tag 'DIFFICULTY' e testing

In [26]:

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Default parameters
MAX_LEN = 500
NUM_WORDS = 20000
BATCH_SIZE = 32
EPOCHS = 10

def preprocess(df, text_columns, label_column, n=10000):
    df = df[text_columns + [label_column]].dropna().head(n).copy()
    df[text_columns[0]] = df[text_columns[0]].apply(eval)
    df[text_columns[1]] = df[text_columns[1]].apply(eval)
    df["full_text"] = df[text_columns[0]].apply(lambda x: " ".join(x)) + " " + df[text_columns[1]].apply(lambda x: " ".join(x))

    label_encoder = LabelEncoder()
    df["encoded_label"] = label_encoder.fit_transform(df[label_column])

    return df, label_encoder

def train_text_classifier(df, label_encoder, model_name_prefix):
    X = df["full_text"]
    y = df["encoded_label"]

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    tokenizer = Tokenizer(num_words=NUM_WORDS, oov_token="<OOV>")
    tokenizer.fit_on_texts(X_train)

    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_val_seq = tokenizer.texts_to_sequences(X_val)

    X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding="post", truncating="post")
    X_val_pad = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding="post", truncating="post")

    model = Sequential([
        Embedding(input_dim=NUM_WORDS, output_dim=128, input_length=MAX_LEN),
        GlobalMaxPooling1D(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(len(label_encoder.classes_), activation='softmax')
    ])

    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(
        X_train_pad, y_train,
        validation_data=(X_val_pad, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[early_stop]
    )

    model.save(f"classifiers/difficulty/{model_name_prefix}.h5")
    with open(f"classifiers/difficulty/{model_name_prefix}_label_mapping.pkl", "wb") as f:
        pickle.dump(dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))), f)

    with open(f"classifiers/difficulty/{model_name_prefix}_tokenizer.pkl", "wb") as f:
        pickle.dump(tokenizer, f)

    print(f"Modello '{model_name_prefix}' salvato con successo.")
    return model

# === SECOND MODEL: DIFFICULTY ===
df_difficulty = df[["INGREDIENTS", "DIRECTIONS", "DIFFICULTY"]].copy()
df_difficulty, difficulty_encoder = preprocess(df_difficulty, ["INGREDIENTS", "DIRECTIONS"], "DIFFICULTY")
difficulty_model = train_text_classifier(df_difficulty, difficulty_encoder, model_name_prefix="difficulty_classifier")

Epoch 1/10


C:\Users\aless\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.5306 - loss: 0.9781 - val_accuracy: 0.6540 - val_loss: 0.7504
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.6609 - loss: 0.7271 - val_accuracy: 0.7010 - val_loss: 0.6436
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 34ms/step - accuracy: 0.7199 - loss: 0.6200 - val_accuracy: 0.7205 - val_loss: 0.6078
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.7570 - loss: 0.5758 - val_accuracy: 0.7185 - val_loss: 0.6058
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.7746 - loss: 0.5215 - val_accuracy: 0.7125 - val_loss: 0.6461
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy: 0.7881 - loss: 0.4950 - val_accuracy: 0.7185 - val_loss: 0.6101
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.8064 - loss: 0.4509 - val_accuracy: 0.7170 - val_loss: 0.6240


Modello 'difficulty_classifier' salvato con successo.


In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

MAX_LEN = 500  # same as training

# Load model and tokenizer
difficulty_model = load_model("classifiers/difficulty/difficulty_classifier.h5")

with open("classifiers/difficulty/difficulty_classifier_tokenizer.pkl", "rb") as f:
    difficulty_tokenizer = pickle.load(f)

with open("classifiers/difficulty/difficulty_classifier_label_mapping.pkl", "rb") as f:
    difficulty_label_map = pickle.load(f)

# Reverse dictionaries to decode predictions
inv_difficulty_label_map = {v: k for k, v in difficulty_label_map.items()}

# List of 10 recipes with expected difficulty
recipes = [
    {
        "ingredients": "Eggs, salt, pepper",
        "instructions": "Beat eggs with salt and pepper. Cook in a pan over medium heat until set.",
        "expected_difficulty": "Easy"
    },
    {
        "ingredients": "Chicken breast, olive oil, garlic, lemon juice, herbs",
        "instructions": "Marinate chicken with garlic, lemon juice, and herbs. Grill until cooked.",
        "expected_difficulty": "Easy"
    },
    {
        "ingredients": "Potatoes, butter, milk, salt, pepper",
        "instructions": "Boil potatoes until tender. Mash with butter and milk. Season with salt and pepper.",
        "expected_difficulty": "Easy"
    },
    {
        "ingredients": "Pasta, tomato sauce, garlic, onion, basil, olive oil",
        "instructions": "Sauté garlic and onion in olive oil. Add tomato sauce and basil. Cook pasta and mix with sauce.",
        "expected_difficulty": "Medium"
    },
    {
        "ingredients": "Beef stew meat, carrots, potatoes, onion, beef broth, red wine, garlic",
        "instructions": "Brown beef in a pot. Add vegetables, broth, and wine. Simmer for 2 hours until tender.",
        "expected_difficulty": "Medium"
    },
    {
        "ingredients": "Salmon fillets, soy sauce, ginger, garlic, honey, sesame oil",
        "instructions": "Marinate salmon with soy sauce, ginger, garlic, honey, and sesame oil. Bake at 180°C for 15 minutes.",
        "expected_difficulty": "Medium"
    },
    {
        "ingredients": "Duck breast, orange juice, honey, thyme, butter",
        "instructions": "Score duck skin and sear in pan. Prepare orange sauce with juice, honey, and thyme. Finish cooking duck and glaze with sauce.",
        "expected_difficulty": "Hard"
    },
    {
        "ingredients": "Lamb shank, rosemary, garlic, red wine, beef broth, carrots, onions",
        "instructions": "Brown lamb shanks. Add vegetables, broth, and wine. Slow cook in oven at 150°C for 3 hours until tender.",
        "expected_difficulty": "Hard"
    },
    {
        "ingredients": "Chocolate, eggs, sugar, butter, flour, cream, gelatin, vanilla bean",
        "instructions": "Melt chocolate and butter. Whisk eggs and sugar. Fold ingredients, bake in water bath at 160°C for 45 minutes. Cool and refrigerate overnight.",
        "expected_difficulty": "Hard"
    },
    {
        "ingredients": "Scallops, bacon, garlic, lemon, parsley, white wine",
        "instructions": "Wrap scallops with bacon and sear. Make sauce with garlic, lemon, parsley, and white wine. Serve scallops with sauce.",
        "expected_difficulty": "Hard"
    }
]

# Prepare input texts for models (ingredients + instructions)
texts = [r["ingredients"] + " " + r["instructions"] for r in recipes]

# Tokenize and padding
diff_sequences = difficulty_tokenizer.texts_to_sequences(texts)
diff_padded = pad_sequences(diff_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

# Predicting difficulty
diff_preds = difficulty_model.predict(diff_padded)
diff_classes = diff_preds.argmax(axis=1)
diff_labels = [inv_difficulty_label_map[c] for c in diff_classes]

# Print results
for i, recipe in enumerate(recipes):
    print(f"Recipe {i+1}:")
    print(f"  Expected difficulty: {recipe['expected_difficulty']}")
    print(f"  Predicted difficulty: {diff_labels[i]}")
    diff_probs_str = ", ".join([f"{inv_difficulty_label_map[j]}: {diff_preds[i][j]*100:.2f}%" for j in range(len(diff_preds[i]))])
    print(f"  Difficulty probabilities: {diff_probs_str}\n")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step
Recipe 1:
  Expected difficulty: Easy
  Predicted difficulty: Easy
  Difficulty probabilities: Easy: 87.47%, Hard: 0.05%, Medium: 12.48%

Recipe 2:
  Expected difficulty: Easy
  Predicted difficulty: Easy
  Difficulty probabilities: Easy: 88.35%, Hard: 0.10%, Medium: 11.55%

Recipe 3:
  Expected difficulty: Easy
  Predicted difficulty: Easy
  Difficulty probabilities: Easy: 76.88%, Hard: 0.32%, Medium: 22.80%

Recipe 4:
  Expected difficulty: Medium
  Predicted difficulty: Easy
  Difficulty probabilities: Easy: 85.94%, Hard: 0.10%, Medium: 13.96%

Recipe 5:
  Expected difficulty: Medium
  Predicted difficulty: Easy
  Difficulty probabilities: Easy: 55.37%, Hard: 0.47%, Medium: 44.16%

Recipe 6:
  Expected difficulty: Medium
  Predicted difficulty: Easy
  Difficulty probabilities: Easy: 70.07%, Hard: 0.30%, Medium: 29.62%

Recipe 7:
  Expected difficulty: Hard
  Predicted difficulty: Medium
  Difficulty probabilities: Easy: 45.84%, Hard: 5.75%, Med

# Creazione del modello per la classificazione del tag 'VEGAN' e testing

In [ ]:

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Default parameters
MAX_LEN = 500
NUM_WORDS = 20000
BATCH_SIZE = 32
EPOCHS = 10

def preprocess(df, text_columns, label_column, n=10000):
    df = df[text_columns + [label_column]].dropna().head(n).copy()
    df[text_columns[0]] = df[text_columns[0]].apply(eval)
    df[text_columns[1]] = df[text_columns[1]].apply(eval)
    df["full_text"] = df[text_columns[0]].apply(lambda x: " ".join(x)) + " " + df[text_columns[1]].apply(lambda x: " ".join(x))

    label_encoder = LabelEncoder()
    df["encoded_label"] = label_encoder.fit_transform(df[label_column])

    return df, label_encoder

def train_text_classifier(df, label_encoder, model_name_prefix):
    X = df["full_text"]
    y = df["encoded_label"]

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    tokenizer = Tokenizer(num_words=NUM_WORDS, oov_token="<OOV>")
    tokenizer.fit_on_texts(X_train)

    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_val_seq = tokenizer.texts_to_sequences(X_val)

    X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding="post", truncating="post")
    X_val_pad = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding="post", truncating="post")

    model = Sequential([
        Embedding(input_dim=NUM_WORDS, output_dim=128, input_length=MAX_LEN),
        GlobalMaxPooling1D(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(len(label_encoder.classes_), activation='softmax')
    ])

    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(
        X_train_pad, y_train,
        validation_data=(X_val_pad, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[early_stop]
    )

    model.save(f"classifiers/vegan/{model_name_prefix}.h5")
    with open(f"classifiers/vegan/{model_name_prefix}_label_mapping.pkl", "wb") as f:
        pickle.dump(dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))), f)

    with open(f"classifiers/vegan/{model_name_prefix}_tokenizer.pkl", "wb") as f:
        pickle.dump(tokenizer, f)

    print(f"Modello '{model_name_prefix}' salvato con successo.")
    return model

# === THIRD MODEL: VEGAN ===
df_vegan = df[["INGREDIENTS", "DIRECTIONS", "VEGAN"]].copy()
df_vegan, vegan_encoder = preprocess(df_vegan, ["INGREDIENTS", "DIRECTIONS"], "VEGAN")
vegan_classifier_model = train_text_classifier(df_vegan, vegan_encoder, model_name_prefix="vegan_classifier")

In [10]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

MAX_LEN = 500  # same as training

# Load model and tokenizer
vegan_model = load_model("classifiers/vegan/vegan_classifier.h5")

with open("classifiers/vegan/vegan_classifier_tokenizer.pkl", "rb") as f:
    vegan_tokenizer = pickle.load(f)

with open("classifiers/vegan/vegan_classifier_label_mapping.pkl", "rb") as f:
    vegan_label_map = pickle.load(f)

# Reverse dictionaries to decode predictions
inv_vegan_label_map = {v: k for k, v in vegan_label_map.items()}

# List of 10 recipes with expected vegan
recipes = [
    {
        "ingredients": "200g spaghetti, 2 cloves garlic, chili pepper, extra virgin olive oil, salt",
        "instructions": "Cook the spaghetti. Meanwhile, sauté garlic and chili in the oil. Mix with the pasta.",
        "expected_vegan": True
    },
    {
        "ingredients": "Lettuce, 1 can of tuna, tomatoes, black olives, onion, olive oil, salt",
        "instructions": "Wash and cut the vegetables. Add the drained tuna and olives. Dress with oil and salt.",
        "expected_vegan": False
    },
    {
        "ingredients": "300g Carnaroli rice, 1 saffron sachet, vegetable broth, butter, onion, Parmesan cheese",
        "instructions": "Sauté the onion, add rice and deglaze with wine. Gradually pour in the broth. Add saffron and stir in butter and cheese.",
        "expected_vegan": False
    },
    {
        "ingredients": "3 eggs, salt, pepper, butter",
        "instructions": "Beat the eggs with salt and pepper. Pour into a pan with melted butter and stir until set.",
        "expected_vegan": False
    },
    {
        "ingredients": "Lasagna sheets, meat sauce, béchamel, Parmesan cheese",
        "instructions": "Layer pasta, sauce, béchamel and cheese. Bake at 180°C for 40 minutes.",
        "expected_vegan": False
    },
    {
        "ingredients": "Apples, bananas, oranges, strawberries, lemon juice, sugar",
        "instructions": "Cut the fruit into pieces, mix with lemon juice and sugar, refrigerate.",
        "expected_vegan": True
    },
    {
        "ingredients": "Chicken breast, onion, curry powder, cream or coconut milk, oil, salt",
        "instructions": "Brown the onion, add diced chicken, curry, and cream. Cook until the chicken is tender.",
        "expected_vegan": False
    },
    {
        "ingredients": "Ladyfingers, mascarpone, eggs, sugar, coffee, cocoa powder",
        "instructions": "Make a cream with yolks, sugar and mascarpone. Layer ladyfingers soaked in coffee and the cream. Dust with cocoa and chill.",
        "expected_vegan": False
    },
    {
        "ingredients": "2 eggs, 250ml milk, 125g flour, sugar, butter",
        "instructions": "Mix flour, eggs and milk into a batter. Cook in pan with butter. Fill as desired.",
        "expected_vegan": False
    },
    {
        "ingredients": "Mixed dried legumes, onion, carrot, celery, vegetable broth, oil, salt",
        "instructions": "Soak legumes for 12 hours. Sauté onion, carrot, celery. Add legumes and broth and cook for at least an hour.",
        "expected_vegan": True
    }
]

# Prepare input texts for models (ingredients + instructions)
texts = [r["ingredients"] + " " + r["instructions"] for r in recipes]

# Tokenize and padding
vegan_sequences = vegan_tokenizer.texts_to_sequences(texts)
vegan_padded = pad_sequences(vegan_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

# Predicting vegan true or false
vegan_preds = vegan_model.predict(vegan_padded)
vegan_classes = vegan_preds.argmax(axis=1)
vegan_labels = [inv_vegan_label_map[c] for c in vegan_classes]

# Print results
for i, recipe in enumerate(recipes):
    print(f"Recipe {i+1}:")
    print(f"  Expected vegan: {recipe['expected_vegan']}")
    print(f"  Predicted vegan: {vegan_labels[i]}")
    vegan_probs_str = ", ".join([f"{inv_vegan_label_map[j]}: {vegan_preds[i][j]*100:.2f}%" for j in range(len(vegan_preds[i]))])
    print(f"  Vegan probabilities: {vegan_probs_str}\n")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
Recipe 1:
  Expected vegan: True
  Predicted vegan: True
  Vegan probabilities: False: 4.01%, True: 95.99%

Recipe 2:
  Expected vegan: False
  Predicted vegan: True
  Vegan probabilities: False: 19.34%, True: 80.66%

Recipe 3:
  Expected vegan: False
  Predicted vegan: False
  Vegan probabilities: False: 99.87%, True: 0.13%

Recipe 4:
  Expected vegan: False
  Predicted vegan: False
  Vegan probabilities: False: 99.95%, True: 0.05%

Recipe 5:
  Expected vegan: False
  Predicted vegan: False
  Vegan probabilities: False: 99.07%, True: 0.93%

Recipe 6:
  Expected vegan: True
  Predicted vegan: True
  Vegan probabilities: False: 3.02%, True: 96.98%

Recipe 7:
  Expected vegan: False
  Predicted vegan: False
  Vegan probabilities: False: 99.99%, True: 0.01%

Recipe 8:
  Expected vegan: False
  Predicted vegan: False
  Vegan probabilities: False: 99.95%, True: 0.05%

Recipe 9:
  Expected vegan: False
  Predicted vegan: False
  Vegan probabilities: Fal

# Creazione del modello per la classificazione del tag 'VEGETARIAN' e testing

In [ ]:

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Default parameters
MAX_LEN = 500
NUM_WORDS = 20000
BATCH_SIZE = 32
EPOCHS = 10

def preprocess(df, text_columns, label_column, n=10000):
    df = df[text_columns + [label_column]].dropna().head(n).copy()
    df[text_columns[0]] = df[text_columns[0]].apply(eval)
    df[text_columns[1]] = df[text_columns[1]].apply(eval)
    df["full_text"] = df[text_columns[0]].apply(lambda x: " ".join(x)) + " " + df[text_columns[1]].apply(lambda x: " ".join(x))

    label_encoder = LabelEncoder()
    df["encoded_label"] = label_encoder.fit_transform(df[label_column])

    return df, label_encoder

def train_text_classifier(df, label_encoder, model_name_prefix):
    X = df["full_text"]
    y = df["encoded_label"]

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    tokenizer = Tokenizer(num_words=NUM_WORDS, oov_token="<OOV>")
    tokenizer.fit_on_texts(X_train)

    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_val_seq = tokenizer.texts_to_sequences(X_val)

    X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding="post", truncating="post")
    X_val_pad = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding="post", truncating="post")

    model = Sequential([
        Embedding(input_dim=NUM_WORDS, output_dim=128, input_length=MAX_LEN),
        GlobalMaxPooling1D(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(len(label_encoder.classes_), activation='softmax')
    ])

    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(
        X_train_pad, y_train,
        validation_data=(X_val_pad, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[early_stop]
    )

    model.save(f"classifiers/vegetarian/{model_name_prefix}.h5")
    with open(f"classifiers/vegetarian/{model_name_prefix}_label_mapping.pkl", "wb") as f:
        pickle.dump(dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))), f)

    with open(f"classifiers/vegetarian/{model_name_prefix}_tokenizer.pkl", "wb") as f:
        pickle.dump(tokenizer, f)

    print(f"Modello '{model_name_prefix}' salvato con successo.")
    return model

# === FOURTH MODEL: VEGETARIAN ===
df_vegetarian = df[["INGREDIENTS", "DIRECTIONS", "VEGETARIAN"]].copy()
df_vegetarian, vegetarian_encoder = preprocess(df_vegetarian, ["INGREDIENTS", "DIRECTIONS"], "VEGETARIAN")
vegetarian_classifier_model = train_text_classifier(df_vegetarian, vegetarian_encoder, model_name_prefix="vegetarian_classifier")

In [11]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

MAX_LEN = 500  # same as training

# Load model and tokenizer
vegetarian_model = load_model("classifiers/vegetarian/vegetarian_classifier.h5")

with open("classifiers/vegetarian/vegetarian_classifier_tokenizer.pkl", "rb") as f:
    vegetarian_tokenizer = pickle.load(f)

with open("classifiers/vegetarian/vegetarian_classifier_label_mapping.pkl", "rb") as f:
    vegetarian_label_map = pickle.load(f)

# Reverse dictionaries to decode predictions
inv_vegetarian_label_map = {v: k for k, v in vegetarian_label_map.items()}

# List of 10 recipes with expected vegetarian
recipes = [
    {
        "ingredients": "200g spaghetti, 2 cloves garlic, chili pepper, extra virgin olive oil, salt",
        "instructions": "Cook the spaghetti. Meanwhile, sauté garlic and chili in the oil. Mix with the pasta.",
        "expected_vegetarian": True
    },
    {
        "ingredients": "Lettuce, 1 can of tuna, tomatoes, black olives, onion, olive oil, salt",
        "instructions": "Wash and cut the vegetables. Add the drained tuna and olives. Dress with oil and salt.",
        "expected_vegetarian": False  # Tuna = non vegetarian
    },
    {
        "ingredients": "300g Carnaroli rice, 1 saffron sachet, vegetable broth, butter, onion, Parmesan cheese",
        "instructions": "Sauté the onion, add rice and deglaze with wine. Gradually pour in the broth. Add saffron and stir in butter and cheese.",
        "expected_vegetarian": True
    },
    {
        "ingredients": "3 eggs, salt, pepper, butter",
        "instructions": "Beat the eggs with salt and pepper. Pour into a pan with melted butter and stir until set.",
        "expected_vegetarian": True
    },
    {
        "ingredients": "Lasagna sheets, meat sauce, béchamel, Parmesan cheese",
        "instructions": "Layer pasta, sauce, béchamel and cheese. Bake at 180°C for 40 minutes.",
        "expected_vegetarian": False  # Meat sauce = non vegetarian
    },
    {
        "ingredients": "Apples, bananas, oranges, strawberries, lemon juice, sugar",
        "instructions": "Cut the fruit into pieces, mix with lemon juice and sugar, refrigerate.",
        "expected_vegetarian": True
    },
    {
        "ingredients": "Chicken breast, onion, curry powder, cream or coconut milk, oil, salt",
        "instructions": "Brown the onion, add diced chicken, curry, and cream. Cook until the chicken is tender.",
        "expected_vegetarian": False  # Chicken = non vegetarian
    },
    {
        "ingredients": "Ladyfingers, mascarpone, eggs, sugar, coffee, cocoa powder",
        "instructions": "Make a cream with yolks, sugar and mascarpone. Layer ladyfingers soaked in coffee and the cream. Dust with cocoa and chill.",
        "expected_vegetarian": True
    },
    {
        "ingredients": "2 eggs, 250ml milk, 125g flour, sugar, butter",
        "instructions": "Mix flour, eggs and milk into a batter. Cook in pan with butter. Fill as desired.",
        "expected_vegetarian": True
    },
    {
        "ingredients": "Mixed dried legumes, onion, carrot, celery, vegetable broth, oil, salt",
        "instructions": "Soak legumes for 12 hours. Sauté onion, carrot, celery. Add legumes and broth and cook for at least an hour.",
        "expected_vegetarian": True
    }
]

# Prepare input texts for models (ingredients + instructions)
texts = [r["ingredients"] + " " + r["instructions"] for r in recipes]

# Tokenize and padding
vegetarian_sequences = vegetarian_tokenizer.texts_to_sequences(texts)
vegetarian_padded = pad_sequences(vegetarian_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

# Predicting vegetarian true or false
vegetarian_preds = vegetarian_model.predict(vegetarian_padded)
vegetarian_classes = vegetarian_preds.argmax(axis=1)
vegetarian_labels = [inv_vegetarian_label_map[c] for c in vegetarian_classes]

# Print results
for i, recipe in enumerate(recipes):
    print(f"Recipe {i+1}:")
    print(f"  Expected vegetarian: {recipe['expected_vegetarian']}")
    print(f"  Predicted vegetarian: {vegetarian_labels[i]}")
    vegetarian_probs_str = ", ".join([f"{inv_vegetarian_label_map[j]}: {vegetarian_preds[i][j]*100:.2f}%" for j in range(len(vegetarian_preds[i]))])
    print(f"  Vegetarian probabilities: {vegetarian_probs_str}\n")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
Recipe 1:
  Expected vegetarian: True
  Predicted vegetarian: True
  Vegetarian probabilities: False: 2.47%, True: 97.53%

Recipe 2:
  Expected vegetarian: False
  Predicted vegetarian: False
  Vegetarian probabilities: False: 93.06%, True: 6.94%

Recipe 3:
  Expected vegetarian: True
  Predicted vegetarian: False
  Vegetarian probabilities: False: 52.48%, True: 47.52%

Recipe 4:
  Expected vegetarian: True
  Predicted vegetarian: True
  Vegetarian probabilities: False: 0.37%, True: 99.63%

Recipe 5:
  Expected vegetarian: False
  Predicted vegetarian: False
  Vegetarian probabilities: False: 52.54%, True: 47.46%

Recipe 6:
  Expected vegetarian: True
  Predicted vegetarian: True
  Vegetarian probabilities: False: 0.11%, True: 99.89%

Recipe 7:
  Expected vegetarian: False
  Predicted vegetarian: False
  Vegetarian probabilities: False: 98.37%, True: 1.63%

Recipe 8:
  Expected vegetarian: True
  Predicted vegetarian: True
  Vegetarian probabiliti

# Creazione del modello per la classificazione del tag 'COOKING_METHOD' e testing

In [10]:

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Default parameters
MAX_LEN = 500
NUM_WORDS = 20000
BATCH_SIZE = 32
EPOCHS = 10

def preprocess(df, text_columns, label_column, n=10000):
    df = df[text_columns + [label_column]].dropna().head(n).copy()
    df[text_columns[0]] = df[text_columns[0]].apply(eval)
    df[text_columns[1]] = df[text_columns[1]].apply(eval)
    df["full_text"] = df[text_columns[0]].apply(lambda x: " ".join(x)) + " " + df[text_columns[1]].apply(lambda x: " ".join(x))

    label_encoder = LabelEncoder()
    df["encoded_label"] = label_encoder.fit_transform(df[label_column])

    return df, label_encoder

def train_text_classifier(df, label_encoder, model_name_prefix):
    X = df["full_text"]
    y = df["encoded_label"]

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    tokenizer = Tokenizer(num_words=NUM_WORDS, oov_token="<OOV>")
    tokenizer.fit_on_texts(X_train)

    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_val_seq = tokenizer.texts_to_sequences(X_val)

    X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding="post", truncating="post")
    X_val_pad = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding="post", truncating="post")

    model = Sequential([
        Embedding(input_dim=NUM_WORDS, output_dim=128, input_length=MAX_LEN),
        GlobalMaxPooling1D(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(len(label_encoder.classes_), activation='softmax')
    ])

    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(
        X_train_pad, y_train,
        validation_data=(X_val_pad, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[early_stop]
    )

    model.save(f"classifiers/method/{model_name_prefix}.h5")
    with open(f"classifiers/method/{model_name_prefix}_label_mapping.pkl", "wb") as f:
        pickle.dump(dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))), f)

    with open(f"classifiers/method/{model_name_prefix}_tokenizer.pkl", "wb") as f:
        pickle.dump(tokenizer, f)

    print(f"Modello '{model_name_prefix}' salvato con successo.")
    return model

# === FIFTH MODEL: METHOD ===
df_method = df[["INGREDIENTS", "DIRECTIONS", "COOKING_METHOD"]].copy()
df_method, method_encoder = preprocess(df_method, ["INGREDIENTS", "DIRECTIONS"], "COOKING_METHOD")
method_classifier_model = train_text_classifier(df_method, method_encoder, model_name_prefix="method_classifier")

C:\Users\aless\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 37ms/step - accuracy: 0.4523 - loss: 1.7384 - val_accuracy: 0.7650 - val_loss: 0.7940
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - accuracy: 0.7853 - loss: 0.7283 - val_accuracy: 0.8660 - val_loss: 0.4808
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 34ms/step - accuracy: 0.8792 - loss: 0.4444 - val_accuracy: 0.9255 - val_loss: 0.3213
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.9122 - loss: 0.3320 - val_accuracy: 0.9340 - val_loss: 0.2616
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.9250 - loss: 0.2618 - val_accuracy: 0.9315 - val_loss: 0.2384
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.9327 - loss: 0.2262 - val_accuracy: 0.9430 - val_loss: 0.2156
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.9402 - loss: 0.2127 - val_accuracy: 0.9495 - val_loss: 0.1973
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.9459 - loss: 0.1780 - val_acc

Modello 'method_classifier' salvato con successo.


In [12]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

MAX_LEN = 500  # same as training

# Load model and tokenizer
method_model = load_model("classifiers/method/method_classifier.h5")

with open("classifiers/method/method_classifier_tokenizer.pkl", "rb") as f:
    method_tokenizer = pickle.load(f)

with open("classifiers/method/method_classifier_label_mapping.pkl", "rb") as f:
    method_label_map = pickle.load(f)

# Reverse dictionaries to decode predictions
inv_method_label_map = {v: k for k, v in method_label_map.items()}

# List of 10 recipes with expected method
recipes = [
    {
        "ingredients": "500g potatoes, olive oil, rosemary, salt",
        "instructions": "Peel and cut the potatoes into chunks. Toss with olive oil and rosemary. Cook in oven at 200°C for 45 minutes.",
        "expected_method": "Roasted"
    },
    {
        "ingredients": "200g spaghetti, salt, water",
        "instructions": "Boil water in a large pot, add salt and cook the spaghetti until al dente.",
        "expected_method": "Boiled"
    },
    {
        "ingredients": "2 chicken breasts, olive oil, herbs, salt",
        "instructions": "Brush chicken with oil and herbs. Cook on a preheated grill for 6 minutes per side.",
        "expected_method": "Grilled"
    },
    {
        "ingredients": "4 eggs, butter, salt, pepper",
        "instructions": "Heat butter in a pan. Crack eggs in and fry until edges are crispy.",
        "expected_method": "Fried"
    },
    {
        "ingredients": "1 salmon fillet, lemon juice, herbs, salt",
        "instructions": "Place salmon in a baking dish, season, and bake at 180°C for 20 minutes.",
        "expected_method": "Baked"
    },
    {
        "ingredients": "Broccoli, carrots, zucchini, salt",
        "instructions": "Wash and cut vegetables. Eat raw or season with salt and olive oil.",
        "expected_method": "Raw"
    },
    {
        "ingredients": "Beef steak, salt, pepper, olive oil",
        "instructions": "Season steak and place under a broiler for 5–7 minutes on each side.",
        "expected_method": "Broil"
    },
    {
        "ingredients": "1 cauliflower, salt, oil, paprika",
        "instructions": "Cut cauliflower into florets. Toss with oil and paprika. Roast in oven at 220°C for 30 minutes.",
        "expected_method": "Roasted"
    },
    {
        "ingredients": "Rice, water, salt",
        "instructions": "Rinse the rice and boil in salted water for 15 minutes.",
        "expected_method": "Boiled"
    },
    {
        "ingredients": "1 can chickpeas, tahini, lemon juice, garlic, olive oil",
        "instructions": "Blend all ingredients in a food processor until smooth. Serve as is.",
        "expected_method": "Other"  # Nessuna cottura classica, solo preparazione a crudo e frullatura
    }
]

# Prepare input texts for models (ingredients + instructions)
texts = [r["ingredients"] + " " + r["instructions"] for r in recipes]

# Tokenize and padding
method_sequences = method_tokenizer.texts_to_sequences(texts)
method_padded = pad_sequences(method_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

# Predicting method
method_preds = method_model.predict(method_padded)
method_classes = method_preds.argmax(axis=1)
method_labels = [inv_method_label_map[c] for c in method_classes]

# Print results
for i, recipe in enumerate(recipes):
    print(f"Recipe {i+1}:")
    print(f"  Expected method: {recipe['expected_method']}")
    print(f"  Predicted method: {method_labels[i]}")
    method_probs_str = ", ".join([f"{inv_method_label_map[j]}: {method_preds[i][j]*100:.2f}%" for j in range(len(method_preds[i]))])
    print(f"  Method probabilities: {method_probs_str}\n")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
Recipe 1:
  Expected method: Roasted
  Predicted method: Other
  Method probabilities: Baked: 3.94%, Blended: 2.66%, Boiled: 1.07%, Broil: 4.60%, Fried: 4.42%, Grilled: 2.63%, Microwaved: 0.64%, Other: 53.05%, Pressure Cooked: 2.72%, Raw: 3.47%, Roasted: 7.69%, Slow Cooking: 0.96%, Steamed: 4.04%, Toasted: 8.10%

Recipe 2:
  Expected method: Boiled
  Predicted method: Boiled
  Method probabilities: Baked: 0.04%, Blended: 0.00%, Boiled: 99.95%, Broil: 0.00%, Fried: 0.00%, Grilled: 0.00%, Microwaved: 0.00%, Other: 0.00%, Pressure Cooked: 0.00%, Raw: 0.00%, Roasted: 0.00%, Slow Cooking: 0.00%, Steamed: 0.00%, Toasted: 0.00%

Recipe 3:
  Expected method: Grilled
  Predicted method: Grilled
  Method probabilities: Baked: 0.01%, Blended: 0.00%, Boiled: 0.07%, Broil: 1.44%, Fried: 1.18%, Grilled: 96.66%, Microwaved: 0.00%, Other: 0.06%, Pressure Cooked: 0.04%, Raw: 0.01%, Roasted: 0.14%, Slow Cooking: 0.23%, Steamed: 0.03%, Toasted: 0.12%

Recipe 4:
  Ex

# Creazione del modello per la classificazione del tag 'price_tag' e testing

In [17]:

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Default parameters
MAX_LEN = 500
NUM_WORDS = 20000
BATCH_SIZE = 32
EPOCHS = 10

# Method to preprocess a dataframe
def preprocess(df, text_columns, label_column, n=10000):
    df = df[text_columns + [label_column]].dropna().head(n).copy()
    df[text_columns[0]] = df[text_columns[0]].apply(eval)
    df[text_columns[1]] = df[text_columns[1]].apply(eval)
    df["full_text"] = df[text_columns[0]].apply(lambda x: " ".join(x)) + " " + df[text_columns[1]].apply(lambda x: " ".join(x))

    label_encoder = LabelEncoder()
    df["encoded_label"] = label_encoder.fit_transform(df[label_column])

    return df, label_encoder

# Method to train a classificator
def train_text_classifier(df, label_encoder, model_name_prefix):
    X = df["full_text"]
    y = df["encoded_label"]

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    tokenizer = Tokenizer(num_words=NUM_WORDS, oov_token="<OOV>")
    tokenizer.fit_on_texts(X_train)

    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_val_seq = tokenizer.texts_to_sequences(X_val)

    X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding="post", truncating="post")
    X_val_pad = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding="post", truncating="post")

    model = Sequential([
        Embedding(input_dim=NUM_WORDS, output_dim=128, input_length=MAX_LEN),
        GlobalMaxPooling1D(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(len(label_encoder.classes_), activation='softmax')
    ])

    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(
        X_train_pad, y_train,
        validation_data=(X_val_pad, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[early_stop]
    )

    # Saving model
    model.save(f"classifiers/price/{model_name_prefix}.h5")
    with open(f"classifiers/price/{model_name_prefix}_label_mapping.pkl", "wb") as f:
        pickle.dump(dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))), f)

    with open(f"classifiers/price/{model_name_prefix}_tokenizer.pkl", "wb") as f:
        pickle.dump(tokenizer, f)

    print(f"Modello '{model_name_prefix}' salvato con successo.")
    return model

# === SIXTH MODEL: price_tag ===
df_price = df[["INGREDIENTS", "DIRECTIONS", "price_tag"]].copy()
df_price, price_encoder = preprocess(df_price, ["INGREDIENTS", "DIRECTIONS"], "price_tag")
price_model = train_text_classifier(df_price, price_encoder, model_name_prefix="price_classifier")

Epoch 1/10


C:\Users\aless\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - accuracy: 0.2718 - loss: 1.5594 - val_accuracy: 0.4255 - val_loss: 1.2593
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 35ms/step - accuracy: 0.4498 - loss: 1.2270 - val_accuracy: 0.5410 - val_loss: 1.0384
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.5099 - loss: 1.0964 - val_accuracy: 0.5890 - val_loss: 0.9638
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.5546 - loss: 1.0165 - val_accuracy: 0.5920 - val_loss: 0.9262
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.5782 - loss: 0.9685 - val_accuracy: 0.6065 - val_loss: 0.9025
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy: 0.6090 - loss: 0.8995 - val_accuracy: 0.6070 - val_loss: 0.8942
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy: 0.6142 - loss: 0.9076 - val_accuracy: 0.6235 - val_loss: 0.8834
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.6273 - loss: 0.8741 - val_accuracy: 0.62

Modello 'price_classifier' salvato con successo.


In [12]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

MAX_LEN = 500  # same as training

# --- Load model and tokenizer ---
price_model = load_model("classifiers/price/price_classifier.h5")

with open("classifiers/price/price_classifier_tokenizer.pkl", "rb") as f:
    price_tokenizer = pickle.load(f)

with open("classifiers/price/price_classifier_label_mapping.pkl", "rb") as f:
    price_label_map = pickle.load(f)

# Reverse dictionaries to decode predictions
inv_price_label_map = {v: k for k, v in price_label_map.items()}

# List of 10 recipes with expected cost category
recipes = [
    {
        "ingredients": "200g spaghetti, 2 cloves garlic, chili pepper, extra virgin olive oil, salt",
        "instructions": "Cook the spaghetti. Meanwhile, sauté garlic and chili in the oil. Mix with the pasta.",
        "expected_cost": "very cheap"
    },
    {
        "ingredients": "Lettuce, 1 can of tuna, tomatoes, black olives, onion, olive oil, salt",
        "instructions": "Wash and cut the vegetables. Add the drained tuna and olives. Dress with oil and salt.",
        "expected_cost": "cheap"
    },
    {
        "ingredients": "300g Carnaroli rice, 1 saffron sachet, vegetable broth, butter, onion, Parmesan cheese",
        "instructions": "Sauté the onion, add rice and deglaze with wine. Gradually pour in the broth. Add saffron and stir in butter and cheese.",
        "expected_cost": "expensive"
    },
    {
        "ingredients": "3 eggs, salt, pepper, butter",
        "instructions": "Beat the eggs with salt and pepper. Pour into a pan with melted butter and stir until set.",
        "expected_cost": "very cheap"
    },
    {
        "ingredients": "Lasagna sheets, meat sauce, béchamel, Parmesan cheese",
        "instructions": "Layer pasta, sauce, béchamel and cheese. Bake at 180°C for 40 minutes.",
        "expected_cost": "expensive"
    },
    {
        "ingredients": "Apples, bananas, oranges, strawberries, lemon juice, sugar",
        "instructions": "Cut the fruit into pieces, mix with lemon juice and sugar, refrigerate.",
        "expected_cost": "cheap"
    },
    {
        "ingredients": "Chicken breast, onion, curry powder, cream or coconut milk, oil, salt",
        "instructions": "Brown the onion, add diced chicken, curry, and cream. Cook until the chicken is tender.",
        "expected_cost": "medium"
    },
    {
        "ingredients": "Ladyfingers, mascarpone, eggs, sugar, coffee, cocoa powder",
        "instructions": "Make a cream with yolks, sugar and mascarpone. Layer ladyfingers soaked in coffee and the cream. Dust with cocoa and chill.",
        "expected_cost": "expensive"
    },
    {
        "ingredients": "2 eggs, 250ml milk, 125g flour, sugar, butter",
        "instructions": "Mix flour, eggs and milk into a batter. Cook in pan with butter. Fill as desired.",
        "expected_cost": "medium"
    },
    {
        "ingredients": "Mixed dried legumes, onion, carrot, celery, vegetable broth, oil, salt",
        "instructions": "Soak legumes for 12 hours. Sauté onion, carrot, celery. Add legumes and broth and cook for at least an hour.",
        "expected_cost": "rich"
    }
]

# Prepare input texts for models (ingredients + instructions)
texts = [r["ingredients"] + " " + r["instructions"] for r in recipes]

# Tokenize and padding
price_sequences = price_tokenizer.texts_to_sequences(texts)
price_padded = pad_sequences(price_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

# Predicting cost tag with probabilities
price_preds = price_model.predict(price_padded)
price_classes = price_preds.argmax(axis=1)
price_labels = [inv_price_label_map[c] for c in price_classes]

# Print results with probabilities
for i, recipe in enumerate(recipes):
    print(f"Recipe {i+1}:")
    print(f"  Expected price tag: {recipe['expected_cost']}")
    print(f"  Predicted price tag: {price_labels[i]}")
    price_probs_str = ", ".join([f"{inv_price_label_map[j]}: {price_preds[i][j]*100:.2f}%" for j in range(len(price_preds[i]))])
    print(f"  Price tag probabilities: {price_probs_str}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
Recipe 1:
  Expected price tag: very cheap
  Predicted price tag: cheap
  Price tag probabilities: cheap: 42.60%, expensive: 7.08%, medium: 27.69%, rich: 0.46%, very cheap: 22.18%
Recipe 2:
  Expected price tag: cheap
  Predicted price tag: cheap
  Price tag probabilities: cheap: 59.37%, expensive: 0.89%, medium: 11.98%, rich: 0.02%, very cheap: 27.75%
Recipe 3:
  Expected price tag: expensive
  Predicted price tag: cheap
  Price tag probabilities: cheap: 52.41%, expensive: 5.25%, medium: 24.34%, rich: 0.27%, very cheap: 17.73%
Recipe 4:
  Expected price tag: very cheap
  Predicted price tag: cheap
  Price tag probabilities: cheap: 50.75%, expensive: 0.19%, medium: 6.28%, rich: 0.00%, very cheap: 42.79%
Recipe 5:
  Expected price tag: expensive
  Predicted price tag: very cheap
  Price tag probabilities: cheap: 24.44%, expensive: 0.18%, medium: 3.15%, rich: 0.00%, very cheap: 72.24%
Recipe 6:
  Expected price tag: cheap
  Predicted price tag: very

# Creazione del modello per la classificazione del tag 'LACTOSE_FREE' e testing

In [13]:

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Default parameters
MAX_LEN = 500
NUM_WORDS = 20000
BATCH_SIZE = 32
EPOCHS = 10

def preprocess(df, text_columns, label_column, n=10000):
    df = df[text_columns + [label_column]].dropna().head(n).copy()
    df[text_columns[0]] = df[text_columns[0]].apply(eval)
    df[text_columns[1]] = df[text_columns[1]].apply(eval)
    df["full_text"] = df[text_columns[0]].apply(lambda x: " ".join(x)) + " " + df[text_columns[1]].apply(lambda x: " ".join(x))

    label_encoder = LabelEncoder()
    df["encoded_label"] = label_encoder.fit_transform(df[label_column])

    return df, label_encoder

def train_text_classifier(df, label_encoder, model_name_prefix):
    X = df["full_text"]
    y = df["encoded_label"]

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    tokenizer = Tokenizer(num_words=NUM_WORDS, oov_token="<OOV>")
    tokenizer.fit_on_texts(X_train)

    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_val_seq = tokenizer.texts_to_sequences(X_val)

    X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding="post", truncating="post")
    X_val_pad = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding="post", truncating="post")

    model = Sequential([
        Embedding(input_dim=NUM_WORDS, output_dim=128, input_length=MAX_LEN),
        GlobalMaxPooling1D(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(len(label_encoder.classes_), activation='softmax')
    ])

    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(
        X_train_pad, y_train,
        validation_data=(X_val_pad, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[early_stop]
    )

    model.save(f"classifiers/lactose_free/{model_name_prefix}.h5")
    with open(f"classifiers/lactose_free/{model_name_prefix}_label_mapping.pkl", "wb") as f:
        pickle.dump(dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))), f)

    with open(f"classifiers/lactose_free/{model_name_prefix}_tokenizer.pkl", "wb") as f:
        pickle.dump(tokenizer, f)

    print(f"Modello '{model_name_prefix}' salvato con successo.")
    return model

# === SEVENTH MODEL: LACTOSE_FREE ===
df_lactose = df[["INGREDIENTS", "DIRECTIONS", "LACTOSE_FREE"]].copy()
df_lactose, lactose_free_encoder = preprocess(df_lactose, ["INGREDIENTS", "DIRECTIONS"], "LACTOSE_FREE")
lactose_free_classifier_model = train_text_classifier(df_lactose, lactose_free_encoder, model_name_prefix="lactose_free_classifier")

Epoch 1/10


C:\Users\aless\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 41ms/step - accuracy: 0.7130 - loss: 0.5332 - val_accuracy: 0.9330 - val_loss: 0.2126
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 41ms/step - accuracy: 0.9294 - loss: 0.2114 - val_accuracy: 0.9450 - val_loss: 0.1756
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.9385 - loss: 0.1870 - val_accuracy: 0.9515 - val_loss: 0.1558
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.9539 - loss: 0.1379 - val_accuracy: 0.9535 - val_loss: 0.1518
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.9586 - loss: 0.1220 - val_accuracy: 0.9525 - val_loss: 0.1540
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.9557 - loss: 0.1161 - val_accuracy: 0.9535 - val_loss: 0.1525
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy: 0.9680 - loss: 0.0914 - val_accuracy: 0.9540 - val_loss: 0.1520


Modello 'lactose_free_classifier' salvato con successo.


In [16]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

MAX_LEN = 500  # same as training

# --- Load model and tokenizer ---
lact_model = load_model("classifiers/lactose_free/lactose_free_classifier.h5")

with open("classifiers/lactose_free/lactose_free_classifier_tokenizer.pkl", "rb") as f:
    lact_tokenizer = pickle.load(f)

with open("classifiers/lactose_free/lactose_free_classifier_label_mapping.pkl", "rb") as f:
    lact_label_map = pickle.load(f)

# Reverse dictionaries to decode predictions
inv_lact_label_map = {v: k for k, v in lact_label_map.items()}

# List of 10 recipes with expected lactose free
recipes = [
    {
        "ingredients": "200g spaghetti, 2 cloves garlic, chili pepper, extra virgin olive oil, salt",
        "instructions": "Cook the spaghetti. Meanwhile, sauté garlic and chili in the oil. Mix with the pasta.",
        "lactose_free": True
    },
    {
        "ingredients": "Lettuce, 1 can of tuna, tomatoes, black olives, onion, olive oil, salt",
        "instructions": "Wash and cut the vegetables. Add the drained tuna and olives. Dress with oil and salt.",
        "lactose_free": True
    },
    {
        "ingredients": "300g Carnaroli rice, 1 saffron sachet, vegetable broth, butter, onion, Parmesan cheese",
        "instructions": "Sauté the onion, add rice and deglaze with wine. Gradually pour in the broth. Add saffron and stir in butter and cheese.",
        "lactose_free": False
    },
    {
        "ingredients": "3 eggs, salt, pepper, butter",
        "instructions": "Beat the eggs with salt and pepper. Pour into a pan with melted butter and stir until set.",
        "lactose_free": False
    },
    {
        "ingredients": "Lasagna sheets, meat sauce, béchamel, Parmesan cheese",
        "instructions": "Layer pasta, sauce, béchamel and cheese. Bake at 180°C for 40 minutes.",
        "lactose_free": False
    },
    {
        "ingredients": "Apples, bananas, oranges, strawberries, lemon juice, sugar",
        "instructions": "Cut the fruit into pieces, mix with lemon juice and sugar, refrigerate.",
        "lactose_free": True
    },
    {
        "ingredients": "Chicken breast, onion, curry powder, cream or coconut milk, oil, salt",
        "instructions": "Brown the onion, add diced chicken, curry, and cream. Cook until the chicken is tender.",
        "lactose_free": False  # assumiamo panna non vegetale
    },
    {
        "ingredients": "Ladyfingers, mascarpone, eggs, sugar, coffee, cocoa powder",
        "instructions": "Make a cream with yolks, sugar and mascarpone. Layer ladyfingers soaked in coffee and the cream. Dust with cocoa and chill.",
        "lactose_free": False
    },
    {
        "ingredients": "2 eggs, 250ml milk, 125g flour, sugar, butter",
        "instructions": "Mix flour, eggs and milk into a batter. Cook in pan with butter. Fill as desired.",
        "lactose_free": False
    },
    {
        "ingredients": "Mixed dried legumes, onion, carrot, celery, vegetable broth, oil, salt",
        "instructions": "Soak legumes for 12 hours. Sauté onion, carrot, celery. Add legumes and broth and cook for at least an hour.",
        "lactose_free": True
    }
]

# Prepare input texts for models (ingredients + instructions)
texts = [r["ingredients"] + " " + r["instructions"] for r in recipes]

# Tokenize and padding
lact_sequences = lact_tokenizer.texts_to_sequences(texts)
lact_padded = pad_sequences(lact_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

# Predicting lactose free with probabilities
lact_preds = lact_model.predict(lact_padded)
lact_classes = lact_preds.argmax(axis=1)
lact_labels = [inv_lact_label_map[c] for c in lact_classes]

# Print results with probabilities
for i, recipe in enumerate(recipes):
    print(f"Recipe {i+1}:")
    print(f"  Expected lactose free: {recipe['lactose_free']}")
    print(f"  Predicted lactose free: {lact_labels[i]}")
    lact_probs_str = ", ".join([f"{inv_lact_label_map[j]}: {lact_preds[i][j]*100:.2f}%" for j in range(len(lact_preds[i]))])
    print(f"  Lactose free probabilities: {lact_probs_str}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Recipe 1:
  Expected lactose free: True
  Predicted lactose free: True
  Lactose free probabilities: False: 12.30%, True: 87.70%
Recipe 2:
  Expected lactose free: True
  Predicted lactose free: True
  Lactose free probabilities: False: 0.54%, True: 99.46%
Recipe 3:
  Expected lactose free: False
  Predicted lactose free: False
  Lactose free probabilities: False: 99.04%, True: 0.96%
Recipe 4:
  Expected lactose free: False
  Predicted lactose free: False
  Lactose free probabilities: False: 93.94%, True: 6.06%
Recipe 5:
  Expected lactose free: False
  Predicted lactose free: False
  Lactose free probabilities: False: 98.65%, True: 1.35%
Recipe 6:
  Expected lactose free: True
  Predicted lactose free: True
  Lactose free probabilities: False: 0.72%, True: 99.28%
Recipe 7:
  Expected lactose free: False
  Predicted lactose free: False
  Lactose free probabilities: False: 93.62%, True: 6.38%
Recipe 8:
  Expected lactose free: False
  Predicted lact

# Creazione del modello per la classificazione del tag 'GLUTEN_FREE' e testing

In [17]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Default parameters
MAX_LEN = 500
NUM_WORDS = 20000
BATCH_SIZE = 32
EPOCHS = 10

def preprocess(df, text_columns, label_column, n=10000):
    df = df[text_columns + [label_column]].dropna().head(n).copy()
    df[text_columns[0]] = df[text_columns[0]].apply(eval)
    df[text_columns[1]] = df[text_columns[1]].apply(eval)
    df["full_text"] = df[text_columns[0]].apply(lambda x: " ".join(x)) + " " + df[text_columns[1]].apply(lambda x: " ".join(x))

    label_encoder = LabelEncoder()
    df["encoded_label"] = label_encoder.fit_transform(df[label_column])

    return df, label_encoder

def train_text_classifier(df, label_encoder, model_name_prefix):
    X = df["full_text"]
    y = df["encoded_label"]

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    tokenizer = Tokenizer(num_words=NUM_WORDS, oov_token="<OOV>")
    tokenizer.fit_on_texts(X_train)

    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_val_seq = tokenizer.texts_to_sequences(X_val)

    X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding="post", truncating="post")
    X_val_pad = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding="post", truncating="post")

    model = Sequential([
        Embedding(input_dim=NUM_WORDS, output_dim=128, input_length=MAX_LEN),
        GlobalMaxPooling1D(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(len(label_encoder.classes_), activation='softmax')
    ])

    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(
        X_train_pad, y_train,
        validation_data=(X_val_pad, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[early_stop]
    )

    model.save(f"classifiers/gluten_free/{model_name_prefix}.h5")
    with open(f"classifiers/gluten_free/{model_name_prefix}_label_mapping.pkl", "wb") as f:
        pickle.dump(dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))), f)

    with open(f"classifiers/gluten_free/{model_name_prefix}_tokenizer.pkl", "wb") as f:
        pickle.dump(tokenizer, f)

    print(f"Modello '{model_name_prefix}' salvato con successo.")
    return model

# === EIGHTH MODEL: GLUTEN_FREE ===
df_gluten = df[["INGREDIENTS", "DIRECTIONS", "GLUTEN_FREE"]].copy()
df_gluten, gluten_free_encoder = preprocess(df_gluten, ["INGREDIENTS", "DIRECTIONS"], "GLUTEN_FREE")
gluten_free_classifier_model = train_text_classifier(df_gluten, gluten_free_encoder, model_name_prefix="gluten_free_classifier")

Epoch 1/10


C:\Users\aless\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 37ms/step - accuracy: 0.8127 - loss: 0.4831 - val_accuracy: 0.8985 - val_loss: 0.2785
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 35ms/step - accuracy: 0.8991 - loss: 0.2366 - val_accuracy: 0.9175 - val_loss: 0.1931
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.9181 - loss: 0.1926 - val_accuracy: 0.9230 - val_loss: 0.1788
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.9287 - loss: 0.1677 - val_accuracy: 0.9220 - val_loss: 0.1739
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.9378 - loss: 0.1438 - val_accuracy: 0.9215 - val_loss: 0.1677
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.9486 - loss: 0.1244 - val_accuracy: 0.9240 - val_loss: 0.1716
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.9538 - loss: 0.1190 - val_accuracy: 0.9215 - val_loss: 0.1707
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy: 0.9597 - loss: 0.1006 - val_accuracy: 0.91

Modello 'gluten_free_classifier' salvato con successo.


In [18]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

MAX_LEN = 500  # same as training

# --- Load model and tokenizer ---
glut_model = load_model("classifiers/gluten_free/gluten_free_classifier.h5")

with open("classifiers/gluten_free/gluten_free_classifier_tokenizer.pkl", "rb") as f:
    glut_tokenizer = pickle.load(f)

with open("classifiers/gluten_free/gluten_free_classifier_label_mapping.pkl", "rb") as f:
    glut_label_map = pickle.load(f)

# Reverse dictionaries to decode predictions
inv_glut_label_map = {v: k for k, v in glut_label_map.items()}

# List of 10 recipes with expected gluten free
recipes = [
    {
        "ingredients": "200g spaghetti, 2 cloves garlic, chili pepper, extra virgin olive oil, salt",
        "instructions": "Cook the spaghetti. Meanwhile, sauté garlic and chili in the oil. Mix with the pasta.",
        "gluten_free": False
    },
    {
        "ingredients": "Lettuce, 1 can of tuna, tomatoes, black olives, onion, olive oil, salt",
        "instructions": "Wash and cut the vegetables. Add the drained tuna and olives. Dress with oil and salt.",
        "gluten_free": True
    },
    {
        "ingredients": "300g Carnaroli rice, 1 saffron sachet, vegetable broth, butter, onion, Parmesan cheese",
        "instructions": "Sauté the onion, add rice and deglaze with wine. Gradually pour in the broth. Add saffron and stir in butter and cheese.",
        "gluten_free": True
    },
    {
        "ingredients": "3 eggs, salt, pepper, butter",
        "instructions": "Beat the eggs with salt and pepper. Pour into a pan with melted butter and stir until set.",
        "gluten_free": True
    },
    {
        "ingredients": "Lasagna sheets, meat sauce, béchamel, Parmesan cheese",
        "instructions": "Layer pasta, sauce, béchamel and cheese. Bake at 180°C for 40 minutes.",
        "gluten_free": False
    },
    {
        "ingredients": "Apples, bananas, oranges, strawberries, lemon juice, sugar",
        "instructions": "Cut the fruit into pieces, mix with lemon juice and sugar, refrigerate.",
        "gluten_free": True
    },
    {
        "ingredients": "Chicken breast, onion, curry powder, cream or coconut milk, oil, salt",
        "instructions": "Brown the onion, add diced chicken, curry, and cream. Cook until the chicken is tender.",
        "gluten_free": True
    },
    {
        "ingredients": "Ladyfingers, mascarpone, eggs, sugar, coffee, cocoa powder",
        "instructions": "Make a cream with yolks, sugar and mascarpone. Layer ladyfingers soaked in coffee and the cream. Dust with cocoa and chill.",
        "gluten_free": False  # ladyfingers contengono farina di frumento
    },
    {
        "ingredients": "2 eggs, 250ml milk, 125g flour, sugar, butter",
        "instructions": "Mix flour, eggs and milk into a batter. Cook in pan with butter. Fill as desired.",
        "gluten_free": False  # contiene farina
    },
    {
        "ingredients": "Mixed dried legumes, onion, carrot, celery, vegetable broth, oil, salt",
        "instructions": "Soak legumes for 12 hours. Sauté onion, carrot, celery. Add legumes and broth and cook for at least an hour.",
        "gluten_free": True
    }
]

# Prepare input texts for models (ingredients + instructions)
texts = [r["ingredients"] + " " + r["instructions"] for r in recipes]

# Tokenize and padding
glut_sequences = glut_tokenizer.texts_to_sequences(texts)
glut_padded = pad_sequences(glut_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

# Predicting gluten free with probabilities
glut_preds = glut_model.predict(glut_padded)
glut_classes = glut_preds.argmax(axis=1)
glut_labels = [inv_glut_label_map[c] for c in glut_classes]

# Print results with probabilities
for i, recipe in enumerate(recipes):
    print(f"Recipe {i+1}:")
    print(f"  Expected gluten free: {recipe['gluten_free']}")
    print(f"  Predicted gluten free: {glut_labels[i]}")
    glut_probs_str = ", ".join([f"{inv_glut_label_map[j]}: {glut_preds[i][j]*100:.2f}%" for j in range(len(glut_preds[i]))])
    print(f"  Gluten free probabilities: {glut_probs_str}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Recipe 1:
  Expected gluten free: False
  Predicted gluten free: False
  Gluten free probabilities: False: 52.81%, True: 47.19%
Recipe 2:
  Expected gluten free: True
  Predicted gluten free: True
  Gluten free probabilities: False: 0.16%, True: 99.84%
Recipe 3:
  Expected gluten free: True
  Predicted gluten free: True
  Gluten free probabilities: False: 1.60%, True: 98.40%
Recipe 4:
  Expected gluten free: True
  Predicted gluten free: True
  Gluten free probabilities: False: 0.40%, True: 99.60%
Recipe 5:
  Expected gluten free: False
  Predicted gluten free: False
  Gluten free probabilities: False: 79.89%, True: 20.11%
Recipe 6:
  Expected gluten free: True
  Predicted gluten free: True
  Gluten free probabilities: False: 0.08%, True: 99.92%
Recipe 7:
  Expected gluten free: True
  Predicted gluten free: True
  Gluten free probabilities: False: 0.03%, True: 99.97%
Recipe 8:
  Expected gluten free: False
  Predicted gluten free: True
  Gluten fr